In [1]:
%%time
import os, re
import pandas as pd
import cx_Oracle
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pyodbc
import psycopg2
from dateutil import parser
pd.set_option('display.max_columns', 116)
#from datetime import datetime
cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\product\11.2.0\instantclient_19_8")
#%load_ext autotime
dsn_tns = cx_Oracle.makedsn('10.96.37.139', '1521', service_name='remdb')
conn_rem = cx_Oracle.connect(user=r'remtmn', password='remtmn', dsn=dsn_tns)
#cursor = conn.cursor()
cursor_rem = conn_rem.cursor()
yesterday=datetime.datetime.today()-datetime.timedelta(days=1)
today=datetime.datetime.today()
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

#from datetime import datetime
def convert_datetime(dt):
    return datetime.datetime.strftime(dt, '%d.%m.%y')
 
conn_sales_ut = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=10.96.64.94;"
                      "Database=SalesUT;"
                      "UID=Sales_user;"
                      "PWD=HknDMZAJ;")
cursor_mssql = conn_sales_ut.cursor()

conn_fl = psycopg2.connect(dbname='flights', user='readonly', 
                        password='igeith4ahfagh2Ga0einei9o', host='db01.mcs-flights.utair.io',application_name='test-app')
cursor_fl = conn_fl.cursor()

Wall time: 5.01 s


In [2]:
%%time
query_min_rbds=f"""SELECT
                    f."number",
                    f."date",
                    f."aircraft",
                    l."departureAirport",
                    l."arrivalAirport", 
                    lah.* 
                        FROM public."Flights" f
                            INNER JOIN public."Legs" l on l."flightId"=f."id"
                            INNER JOIN 
                            (SELECT 
                                la."legId",
                                la."available",
                                la."booked",
                                la."updated",
                                la."state",
                                ca."code" as cabin,
                                ca."available" as ca_av,
                                ca."booked" as ca_bkd,
                                ca."waitlist" as ca_wait,
                                sca."code" as RBD,
                                sca."state" as rbd_state,
                                sca."available" as rbd_av,
                                sca."booked" rbd_bkd,
                                sca."waitlist" rbd_wait
                                    from 
                                    (SELECT DISTINCT ON ("legId") * 
                                        FROM public."LegAvailabilityHistory"  
                                            WHERE 
                                                "updated">=now()::date-14 
                                            ORDER BY 
                                                "legId","updated" DESC) as la
                                    INNER JOIN  "CabinAvailabilityHistory" ca 
                                        on la."id"=ca."LegAvailabilityId"
                                    INNER JOIN "SubclassAvailabilityHistory" sca 
                                        on sca."CabinAvailabilityId"=ca."id") as lah
                            on lah."legId"=l."id"
                                WHERE 
                                f."date">=now()::date and f."date"<=now()::date+91 and f.number<2000"""

df_rbd = pd.read_sql(query_min_rbds, con=conn_fl)

Wall time: 2min 11s


In [3]:
# %%time
# query_cities=f"""SELECT c."code", c."name"  from public."Cities" c"""
# df_city = pd.read_sql(query_cities, con=conn_fl)
# df_city['name']=df_city['name'].apply(pd.Series)['ru']

 # Rcnt bookings by rbd

In [4]:
# %%time
# rcnt_with_code=f"""
# SELECT
#     f. "date",
#     f. "number",
#     l."departureAirport",
#     l."arrivalAirport", 
#     rcnt. "hours",
#     rcnt. "rcnt" AS rcnt,
#     rcnt. "last_updated",
#     rcnt."last_code"
# FROM (
#     SELECT
#         random_name. "last_updated",
#         random_name. "last_legId",
#         random_name."last_code",
#         (random_name. "last_booked" - random_name. "notlast_booked") AS rcnt,
#         (DATE_PART('day', random_name. "last_updated"::timestamp - random_name. "notlast_updated"::timestamp) * 24 + DATE_PART('hour', random_name. "last_updated"::timestamp - random_name. "notlast_updated"::timestamp)) AS hours
#     FROM (SELECT *
#           FROM(SELECT sch2."code" AS  "notlast_code",
#                       sch2."booked" AS "notlast_booked",
#                       lah2."updated" AS "notlast_updated",
#                       lah2."legId" AS "notlast_legId"
#                 FROM public."LegAvailabilityHistory" lah2
#                         LEFT JOIN public."CabinAvailabilityHistory" cah2
#                           ON cah2."LegAvailabilityId"=lah2."id"
#                         LEFT JOIN public."SubclassAvailabilityHistory" sch2
#                           ON sch2."CabinAvailabilityId"=cah2."id"
#                           WHERE 
#                           lah2.state = 'open' and lah2."updated" >= now()::date - 4 and sch2."code" not like 'Q') lah
#                       LEFT JOIN ( SELECT DISTINCT ON ("last_legId", "last_code") 
#                                   sch1."code" AS  "last_code",
#                                   sch1."booked" AS "last_booked",
#                                   lah1."updated" AS "last_updated",
#                                   lah1."legId" AS "last_legId"
#                                   --lah.booked, sch.legId, lah.updated 
#                                   FROM public."LegAvailabilityHistory" lah1
#                                   LEFT JOIN public."CabinAvailabilityHistory" cah1 
#                                     ON cah1."LegAvailabilityId"=lah1."id"
#                                   LEFT JOIN public."SubclassAvailabilityHistory" sch1 
#                                     ON sch1."CabinAvailabilityId"=cah1."id"
#                                     WHERE 
#                                     lah1.state = 'open' and lah1."updated" >= now()::date - 1 and sch1."code" not like 'Q'
#                                     ORDER BY "last_legId","last_code","last_updated" DESC) lastlah 
#                                     ON (lastlah."last_legId" = lah."notlast_legId" AND lastlah."last_code"=lah."notlast_code")) random_name) rcnt
#     LEFT JOIN public. "Legs" l ON l. "id" = rcnt. "last_legId"
#     LEFT JOIN public. "Flights" f ON f. "id" = l. "flightId"
#     --LEFT JOIN public. "CabinAvailabilityHistory" cabin ON rcnt. "id" = l. "flightId"
# WHERE
#     f. "date" >= now()::date
#     AND f. "date" <= now()::date + 90
#     AND rcnt. "hours" > 21
#     AND rcnt. "hours" < 77"""

# df_rcnt_with_code = pd.read_sql(rcnt_with_code, con = conn_fl)

# next7days = datetime.date.today () + datetime.timedelta(days = 7)

In [5]:
#next7days

In [6]:
# df_rcnt_with_code24 = df_rcnt_with_code[(df_rcnt_with_code['hours'] >= 22) & (df_rcnt_with_code['hours'] <= 26) & (df_rcnt_with_code['date'] < next7days)].groupby (['date', 'number', 'last_updated','last_code','arrivalAirport','departureAirport']).agg ({'rcnt' :'mean' }).reset_index()
# df_rcnt_with_code72 = df_rcnt_with_code[(df_rcnt_with_code['hours'] >= 68) & (df_rcnt_with_code['hours'] <= 76) & (df_rcnt_with_code['date'] > next7days)].groupby (['date', 'number', 'last_updated','last_code','arrivalAirport','departureAirport']).agg ({'rcnt' :'mean' }).reset_index()
# df_rcnt_with_code1 = pd.concat([df_rcnt_with_code24, df_rcnt_with_code72]).reset_index()

# # Добавил в комментарий из за потери информации о маршруте в дальнейшем всплывают значения NaN 15.06.21
# # df_rcnt_with_code1 = df_rcnt_with_code1[df_rcnt_with_code1['rcnt']!=0]

# End rcnt bookings by rbd

In [7]:
# %%time
# df_city.to_csv('Code_Dict.csv',sep=';',index=False,encoding='utf-8-sig')

In [8]:
%%time
rcnt=f""" SELECT
            f. "date",
            f. "number",
            rcnt. "hours",
            rcnt. "rcnt" AS rcnt,
            rcnt. "updated"
        FROM (
            SELECT
                lastlah. "updated",
                lastlah. "legId",
                (lastlah. "booked" - lah. "booked") AS rcnt,
                (DATE_PART('day', lastlah. "updated"::timestamp - lah. "updated"::timestamp) * 24 + DATE_PART('hour', lastlah. "updated"::timestamp - lah. "updated"::timestamp)) AS hours
            FROM
                public. "LegAvailabilityHistory" lah
            LEFT JOIN ( SELECT DISTINCT ON ("legId")
                    *
                FROM
                    public. "LegAvailabilityHistory"
                WHERE
                    state = 'open' and "updated" >= now()::date - 1
                ORDER BY
                    "legId",
                    "updated" DESC) lastlah ON lastlah. "legId" = lah. "legId"
            WHERE
               lah.state = 'open' and  lah. "updated" >= now()::date - 4
            ORDER BY
                lah. "legId",
                lah. "updated" DESC) rcnt
            LEFT JOIN public. "Legs" l ON l. "id" = rcnt. "legId"
            LEFT JOIN public. "Flights" f ON f. "id" = l. "flightId"
        WHERE
            f. "date" >= now()::date
            AND f. "date" <= now()::date + 90
            AND rcnt. "hours" > 21
            AND rcnt. "hours" < 77 """
df_rcnt = pd.read_sql(rcnt, con = conn_fl) 
next7days = datetime.date.today () + datetime.timedelta (days = 7)
df_rcnt24 = df_rcnt[(df_rcnt['hours'] >= 22) & (df_rcnt['hours'] <= 26) & (df_rcnt['date'] < next7days)].groupby (['date', 'number', 'updated']).agg ({'rcnt' :'mean' }).reset_index()
df_rcnt72 = df_rcnt[(df_rcnt['hours'] >= 68) & (df_rcnt['hours'] <= 76) & (df_rcnt['date'] > next7days)].groupby (['date', 'number', 'updated']).agg ({'rcnt' :'mean' }).reset_index()
df_rcnt = pd.concat([df_rcnt24, df_rcnt72]).reset_index ()

Wall time: 23.2 s


In [9]:
# df_rcnt[df_rcnt['number']<400].sort_values(by=['rcnt'],ascending=False)

In [10]:
%%time
query_rev="""SELECT 
                [Номер рейса],
                [Пункт вылета],
                [Пункт прилета],
                [Дата вылета],
                
                (sum([Тариф на купоне РУБ]) + sum([Топливный сбор РУБ]) + sum([Штраф за возврат\обмен (CP) РУБ])+
                 sum([Сбор RU РУБ]) + SUM([Сбор за бланк (PS,YR) РУБ]) + sum([Прочие сборы РУБ]) -
                 SUM([Сумма бонусов на купоне (эквив. РУБ)])) as 'rev_tct',
                
                sum([Количество проданных сегментов]) as 'Pax'
            FROM data2020view 
            WHERE
                [Дата вылета] >= getdate() and  [Дата вылета]<dateadd(DD,90,getdate()) and
                [Вид тарифа] not like '%REG%' and [Вид тарифа] not like '%CHARTER%' and
                [Перевозчик]='UT' and [Участвует в обмене?]='No' and [Вид операции (бланка)]='TCT'
            GROUP BY 
                [Номер рейса],
                [Пункт вылета],
                [Пункт прилета],
                [Дата вылета]"""
    
df_sales=pd.read_sql(query_rev,con=conn_sales_ut)
df_sales['depdate']=df_sales['Дата вылета'].apply(convert_datetime)

df_sales=df_sales.groupby(['Номер рейса','Пункт вылета','Пункт прилета','depdate']).agg({'rev_tct':'sum','Pax':'sum'}).reset_index()

df_sales['avg_fare']=round(np.divide(df_sales['rev_tct'],df_sales['Pax']),0)
df_sales['fl_num']=df_sales['Номер рейса'].astype(int)

Wall time: 1min 55s


In [11]:
%%time
#booking curves prev month

#get history of bookings by snap_nr for last month
query_min_fares=f"""SELECT 
  lh.FLIGHT_NR,lh.DEP_DATE, lh.ORIGIN,lh.DEST,
  lh.EQUIP_ID,sd.SNAP_NR,sc.DAYS_TO_DEP,
  SUM(sd.ACT_BKD) "ACT_BKD",
  SUM(sd.ACT_GRP_BKD) "GRP_BKD", 
  SUM(sd.CALC_FRC_MIXED) "CALC_FRC_MIXED",--SUM(ls.CALC_FRC_FLOWN) "CALC_FRC_FLOWN",
  avg(lh.LEG_CAP) "CAP", 
  (case when  avg(lh.LEG_CAP) <> 0 then round(SUM(sd.CALC_FRC_MIXED)/avg(lh.LEG_CAP)*100,0) end) "FLF", 
  (case when  avg(lh.LEG_CAP) <> 0 then round((SUM(sd.ACT_BKD)+SUM(sd.ACT_GRP_BKD))/avg(lh.LEG_CAP)*100,0) end) "BLF"
 FROM LEG_HEADER lh

LEFT JOIN SEGMENT_HEADER sh
  ON sh.FLIGHT_KEY=lh.FLIGHT_KEY AND sh.ORIGIN=lh.ORIGIN AND sh.DEST=lh.dest
LEFT JOIN SEGMENT_DETAILS sd
  ON sh.SEG_KEY=sd.SEG_KEY
  LEFT JOIN FLIGHT f
  ON f.FLIGHT_KEY=lh.FLIGHT_KEY
LEFT JOIN SNAPSHOT_CONTROL sc
  ON sc.SNAP_NR=sd.SNAP_NR AND sc.SNAP_ID=f.SNAP_ID
  WHERE lh.DEP_DATE>= SYSDATE-30 AND lh.DEP_DATE<=SYSDATE  AND lh.LAST_SNAP_NR IS NOT null

  -- lh.DEP_DATE=TO_DATE('2021/03/12', 'yyyy/mm/dd') and lh.FLIGHT_NR=137
  GROUP BY  
  
  lh.FLIGHT_NR,lh.DEP_DATE, lh.ORIGIN,lh.DEST,
  lh.EQUIP_ID,sd.SNAP_NR,sc.DAYS_TO_DEP"""

df = pd.read_sql(query_min_fares, con=conn_rem)

Wall time: 2min 47s


In [12]:

%%time
#booking curves for 2019 year

#get history of bookings by snap_nr for last month
query=f"""--booking curves for 2019 year
SELECT q.FLIGHT_NR,to_number(TO_CHAR(q.DEP_DATE,'MM')) "month",q.ORIGIN,q.DEST,
  q.SNAP_NR,q.DAYS_TO_DEP,
  (case when  sum(q.CAP) <> 0 then round(SUM(q.CALC_FRC_MIXED)/sum(q.CAP),2) end) "FLF", 
  (case when  sum(q.CAP) <> 0 then round((SUM(q.ACT_BKD))/sum(q.CAP),2) end) "BLF"
 FROM 
(SELECT 
  lh.FLIGHT_NR,lh.DEP_DATE, lh.ORIGIN,lh.DEST,sd.SNAP_NR,sc.DAYS_TO_DEP,
  round(SUM(sd.ACT_BKD),0) "ACT_BKD",
  round(SUM(sd.ACT_GRP_BKD),0) "GRP_BKD", 
  round(SUM(sd.CALC_FRC_MIXED),0) "CALC_FRC_MIXED",--SUM(ls.CALC_FRC_FLOWN) "CALC_FRC_FLOWN",
  round(avg(lh.LEG_CAP),0) "CAP"
 -- (case when  avg(lh.LEG_CAP) <> 0 then round(SUM(sd.CALC_FRC_MIXED)/avg(lh.LEG_CAP)*100,0) end) "FLF", 
 -- (case when  avg(lh.LEG_CAP) <> 0 then round((SUM(sd.ACT_BKD))/avg(lh.LEG_CAP)*100,0) end) "BLF"
 FROM LEG_HEADER lh

LEFT JOIN SEGMENT_HEADER sh
  ON sh.FLIGHT_KEY=lh.FLIGHT_KEY AND sh.ORIGIN=lh.ORIGIN AND sh.DEST=lh.dest
LEFT JOIN SEGMENT_DETAILS sd
  ON sh.SEG_KEY=sd.SEG_KEY
  LEFT JOIN FLIGHT f
  ON f.FLIGHT_KEY=lh.FLIGHT_KEY
LEFT JOIN SNAPSHOT_CONTROL sc
  ON sc.SNAP_NR=sd.SNAP_NR AND sc.SNAP_ID=f.SNAP_ID
  WHERE lh.DEP_DATE>= SYSDATE-365*2 AND lh.DEP_DATE<=SYSDATE+90-365*2  AND lh.LAST_SNAP_NR IS NOT NULL AND sd.SNAP_NR>0

 -- and lh.FLIGHT_NR=329
  GROUP BY  
  
  lh.FLIGHT_NR,lh.DEP_DATE, 
  lh.ORIGIN,lh.DEST,
 sd.SNAP_NR,sc.DAYS_TO_DEP) q
--WHERE to_number(TO_CHAR(q.DEP_DATE,'MM'))=5
  GROUP BY
q.FLIGHT_NR,to_number(TO_CHAR(q.DEP_DATE,'MM')),q.ORIGIN,q.DEST,
  q.SNAP_NR,q.DAYS_TO_DEP
"""

df19 = pd.read_sql(query, con=conn_rem)

Wall time: 1min 13s


In [13]:
%%time
#booking curves

def getaircrafttype(row):
    if row['CAP']<48:
        return "AN4"
    elif row['CAP']<75:
        return "AT7"
    elif row['CAP']<140:
        return 735
    elif row['CAP']<160:
        return 734
    elif row['CAP']<220:
        return 738
    elif row['CAP']>=220:
        return 762
    else:
        return 0
df['aircraft type']= df.apply(getaircrafttype, axis=1)


dict_cap={'762':249,'735':116,'738':176,'734':150,"AT7":70}

df['Sale_date']=df['DEP_DATE']-df['DAYS_TO_DEP'].apply(lambda x: pd.Timedelta(x, unit='D'))
df['od']=df[['ORIGIN','DEST']].apply(lambda x: '-'.join(sorted(x)),axis=1)
df['last_cap']=df['EQUIP_ID'].map(dict_cap)
df['lf_last_cap']=df['ACT_BKD']/df['last_cap']
df['lf_act_cap']=df['ACT_BKD']/df['CAP']
df['month']=pd.DatetimeIndex(df['DEP_DATE']).month
df['weeknum']=df['DEP_DATE'].dt.week
#df['lf_by_last_cap'] = df['lf_by_last_cap'].astype(float).map("{:.1%}".format)
#df['lf_by_act_cap']=df['lf_by_act_cap'].astype(float).map("{:.1%}".format)
#booking curves
#df_c=df.groupby(['FLIGHT_NR','ORIGIN','DEST','od','month','DAYS_TO_DEP','SNAP_NR']).agg({'lf_last_cap':['mean','median','std',percentile(75),percentile(25)]}).reset_index()
df_c=df.groupby(['FLIGHT_NR','ORIGIN','DEST','od','DAYS_TO_DEP','SNAP_NR']).agg({'lf_last_cap':['mean','median','std',percentile(75),percentile(25)]}).reset_index()
df_c.columns=df_c.columns.map('.'.join).str.strip('.')

<timed exec>:29: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


Wall time: 4.74 s


In [14]:
%%time
# Infare


# miscellaneous values for data processing

columns_source_dict = {
    'cxr': 'carrier',
    'orig atp': 'origin',
    'dest apt': 'destination',
    #'first fcc': 'farebasis',
    'first flt#': 'flight_no',
    'first flt date': 'flight_datetime',
    'inclusive': 'price',
    'shapshot': 'snapshot',
    'pos': 'pos',
    'tax':'tax',
    'source':'source',
    #'source': 'source',
    #'out_booking_class': 'rbd'
}

columns_empty_always = [
    'season', 'advpur', 'minstay', 
    'maxstay', 'first ticket', 'last ticket',
    'first travel', 'last travel', 'penalty',
    'homebound_travel_stop_over', 'out_booking_class','source','first fcc',
    'home_booking_class'
]

columns_empty_ow = ['last flt#', 'last flt date', 'last fcc']
columns_empty_direct = ['outbound_travel_stop_over']

# mow_airports = ['VKO', 'SVO', 'DME', 'ZIA']

airport_to_cities_dic = {
    'SVO': 'VKO', 'DME': 'VKO', 'ZIA': 'VKO', 
    'SXF': 'TXL',
    'BGY': 'MXP',
    'SAW':'IST'
}

def replace_airports_to_cities(text):
    for i, j in airport_to_cities_dic.items():
        text = text.replace(i, j)
    return text

# process single infare file into all carriers prices 

def process_df2(df, date_taken):
    df = df[(df['fty']=='OW') & (df['outbound_travel_stop_over'].isna())]

    columns_empty = columns_empty_always + columns_empty_direct + columns_empty_ow
    #assert df[columns_empty].isna().all().all()
    df = df[columns_source_dict].rename(columns=columns_source_dict)

    #     assert (df['carrier'] == df['flight_no'].apply(lambda x: x[:2])).all()
    df = df[df['carrier'] == df['flight_no'].apply(lambda x: x[:2])].copy()

    df['flight_no'] = df['flight_no'].apply(lambda x: x[2:])
    df['price'] = df['price'].apply(lambda x: int(float(x)))

    df['snapshot'] = df['snapshot'].apply(lambda x: x[:10])
    df['flight_date'] = df['flight_datetime'].apply(lambda x: x[:10])
    df['departure_time']= df['flight_datetime'].apply(lambda x: x[11:])
    df['o'] = df['origin'].apply(replace_airports_to_cities)
    df['d'] = df['destination'].apply(replace_airports_to_cities)
    df['od']=df[['o','d']].apply(lambda x: ''.join(sorted(x)),axis=1)

    return df


# In[5]:


# config 
# change this to process other dates 
#\\10.96.37.180\infare\ftp.infare.net\LSYRembrandtFormat\Newest
data_path = '//10.96.37.180/infare/ftp.infare.net/LSYRembrandtFormat/Newest'


# In[6]:
yesterday_file='UT_data'+datetime.datetime.strftime(datetime.datetime.today()-datetime.timedelta(days=1),'%Y%m%d')
today='UT_'+datetime.datetime.strftime(datetime.datetime.today(),'%Y%m%d')+'_0900'
dates=[yesterday_file,today]

# combine multiple dates taken into one mean prices dataframe 
# can swap process_df <---> process_df_all_carriers

dfs = []
try:
    for date_taken in dates:
        df_Infare = pd.read_csv(os.path.join(data_path, f'{date_taken}.csv'), sep=';', low_memory=False)
        dfs.append(process_df2(df_Infare,date_taken))
except OSError:
    pass
df_Infare = pd.concat(dfs).reset_index()

dfI=df_Infare.drop_duplicates().sort_values(['carrier','flight_no','flight_datetime','snapshot']).reset_index(drop=True)




df_UT=dfI[(dfI.carrier=='UT')  & (dfI['source'] != 'aviasales')]
df_Other=dfI[dfI.carrier!='UT']
df_Other=df_Other.drop_duplicates(['carrier','flight_no','flight_datetime'], keep='last')
df_UT=df_UT.drop_duplicates(['carrier','flight_no','flight_datetime'], keep='last')

Wall time: 51.3 s


In [15]:
%%time

# Tax_inf - dataframe with info about tax
tax_inf = df_UT[['origin', 'destination', 'tax','price','flight_datetime','flight_no']].\
groupby(['origin', 'destination','tax','price','flight_datetime','flight_no']).\
count().reset_index().rename(columns = {'tax':'infare_tax'})

tax_inf_2 = df_UT[['origin', 'destination', 'tax']].\
groupby(['origin', 'destination','tax']).\
count().reset_index().rename(columns = {'tax':'infare_tax'})

#  Drop duplicate tax on direction
tax_inf_2['od'] = tax_inf_2[['origin','destination']].apply(lambda x: x[0] + x[1],axis=1)
tax_inf_2 = tax_inf_2.sort_values('infare_tax').drop_duplicates(['od'],keep='last')


tax_inf = tax_inf.merge(tax_inf_2, on = ['origin','destination'])
tax_inf['flight_datetime'] = tax_inf['flight_datetime'].apply(lambda x: parser.parse(x, dayfirst=True))
tax_inf['flight_no'] = tax_inf['flight_no'].astype(int)


Wall time: 1.06 s


In [16]:
%%time
#take min prices by fare_groups, o, d,flight_date
minfg=['R3','DP','5N','A4','ZF','WZ','U6','N4','Y7','PC'] #Якутия,Победа,Нордавиа,Азимут,Азур,РедВингс,Урал,Нордвинд,Нордстар,Пегас
avgfg=['S7','6R','YC','IO','I8','7R','D2','B2']      #Сибирь,Алроса,Ямал,Ираэро,Ижавиа,Руслайн,Белавиа
maxfg=['SU']                               #АФЛ

def getfaregroup(row):
    if row['carrier'] in minfg:
        return "min"
    elif row['carrier'] in avgfg:
        return "avg"
    elif row['carrier'] in maxfg:
        return "max"
    else:
        return 0
    
df_fg=df_Other    
df_fg['fare_group']= df_fg[['carrier']].apply(lambda x: getfaregroup(x), axis=1)

min_fg=df_fg[df_fg['fare_group']=='min'].groupby(['o','d','flight_date']).agg({'price':['min','median','max']}).reset_index()
avg_fg=df_fg[df_fg['fare_group']=='avg'].groupby(['o','d','flight_date']).agg({'price':['min','median','max']}).reset_index()
max_fg=df_fg[df_fg['carrier']=='SU'].groupby(['o','d','flight_date']).agg({'price':['min','median','max']}).reset_index()

min_fg.columns=min_fg.columns.map('.'.join).str.strip('.')
avg_fg.columns=avg_fg.columns.map('.'.join).str.strip('.')
max_fg.columns=max_fg.columns.map('.'.join).str.strip('.')


min_fg=min_fg.rename(columns={'price.min':'min'})
avg_fg=avg_fg.rename(columns={'price.min':'mean'})
max_fg=max_fg.rename(columns={'price.min':'max'})

df_InfareFG=df_UT.merge(min_fg[['min','o','d','flight_date']],left_on=['o','d','flight_date'],right_on=['o','d','flight_date'],how='left')
df_InfareFG=df_InfareFG.merge(avg_fg[['mean','o','d','flight_date']],left_on=['o','d','flight_date'],right_on=['o','d','flight_date'],how='left')
df_InfareFG=df_InfareFG.merge(max_fg[['max','o','d','flight_date']],left_on=['o','d','flight_date'],right_on=['o','d','flight_date'],how='left')


df_InfareFG['DAYS_TO_DEP2']=pd.to_datetime(df_InfareFG['flight_date'],format='%Y-%m-%d')-pd.to_datetime(df_InfareFG['snapshot'],format='%Y-%m-%d')
df_InfareFG['flight_date']=pd.to_datetime(df_InfareFG['flight_date'],format='%Y-%m-%d')
df_InfareFG['DAYS_TO_DEP']=df_InfareFG['DAYS_TO_DEP2'].astype('str').str.split(' ').str[0]
df_InfareFG['DAYS_TO_DEP']=df_InfareFG['DAYS_TO_DEP'].astype('int64')
df_InfareFG['snapshot']=pd.to_datetime(df_InfareFG['snapshot'])
df_InfareFG['flight_no']=df_InfareFG['flight_no'].astype('int64')
df_InfareFG=df_InfareFG.rename(columns={'price':'UT_price_infare'})

Wall time: 876 ms


In [17]:
%%time
query=f"""SELECT * FROM FARE_INPUT"""
df_sirena=pd.read_sql(query,con=conn_rem)

filepath = '//utair.dom/ut/UTPA_OUD/Запорощенко/'

#YQ=pd.read_excel(filepath+'Сбор YQ.xlsx',sep=',',sheet_name='data')
YQ=pd.read_excel(filepath+'Сбор YQ.xlsx', sheet_name='data')
YQ['od']=YQ['od'].str[:3]+""+YQ['od'].str[-3:]

Wall time: 15.3 s


In [18]:
# Add YQ_tax to df tax_inf
tax_inf = tax_inf.merge(YQ[['sum_manual','o','d']], left_on=['origin','destination'],right_on=['o','d'],how='left').\
drop(columns=['o','d']).rename(columns={'sum_manual':'YQ_tax'})

In [19]:
%%time
tax_inf.to_excel('tax_inf.xlsx')

Wall time: 2.96 s


In [20]:
%%time
dict_RBD={
    'F':26,'Z':25, 'P':24,'K':23, 'L':22, 'H':21, 'V':20, 'R':19, 'O':18, 'U':17, 'W':16, 'X':15, 'B':14,
    'N':13, 'G':12, 'Q':11, 'E':10, 'T':9, 'S':8, 'Y':7, 'A':5, 'D':4, 'I':3, 'J':2, 'C':1
}
df_sirena['numRBD']=df_sirena['CLASS_NAME'].map(dict_RBD)
df_sirena['od']=df_sirena[['ORIGIN','DEST']].apply(lambda x: '-'.join(sorted(x)),axis=1)

df_sirena=df_sirena[~df_sirena['CLASS_NAME'].isin(['C','J','I','D','A'])]

df_sirena=df_sirena.sort_values(['od','ORIGIN','DEST','numRBD','DEP_DATE_FROM','BOOKING_DATE_FROM'])
# df_sirena=df_sirena.drop_duplicates(['od','ORIGIN','DEST','numRBD'], keep='first')

    
df_sirena['fare-1'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=-1)
df_sirena['fare-2'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=-2)
df_sirena['fare-3'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=-3)
df_sirena['fare+1'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=1)
df_sirena['fare+2'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=2)
df_sirena['fare+3'] = df_sirena.groupby(['od','ORIGIN','DEST'])['FARE'].shift(periods=3)

df_sirena['fare-1'] = df_sirena[['FARE','fare-1']].apply(lambda x: 'min' if np.isnan(x['fare-1']) else x['fare-1']-x['FARE'],axis=1)
df_sirena['fare-2'] = df_sirena[['FARE','fare-2']].apply(lambda x: 'min' if np.isnan(x['fare-2']) else x['fare-2']-x['FARE'],axis=1)
df_sirena['fare-3'] = df_sirena[['FARE','fare-3']].apply(lambda x: 'min' if np.isnan(x['fare-3']) else x['fare-3']-x['FARE'],axis=1)
df_sirena['fare+1'] = df_sirena[['FARE','fare+1']].apply(lambda x: 'max' if np.isnan(x['fare+1']) else x['fare+1']-x['FARE'],axis=1)
df_sirena['fare+2'] = df_sirena[['FARE','fare+2']].apply(lambda x: 'max' if np.isnan(x['fare+2']) else x['fare+2']-x['FARE'],axis=1)
df_sirena['fare+3'] = df_sirena[['FARE','fare+3']].apply(lambda x: 'max' if np.isnan(x['fare+3']) else x['fare+3']-x['FARE'],axis=1)


df_sirena=df_sirena.merge(YQ[['o','d','sum_manual','type']], left_on=['ORIGIN','DEST'], right_on=['o','d'],how='left')

# Add feature UT_price(if tax is not exist UT_price is sirena`s FARE else FARE+tax)
df_sirena['UT_price']=df_sirena.apply(lambda x: x['FARE'] if np.isnan(x['sum_manual']) else x['sum_manual']+x['FARE'],axis=1)
df_sirena['tax']=df_sirena.apply(lambda x: 'fare' if np.isnan(x['sum_manual']) else 'tax+fare',axis=1)

Wall time: 704 ms


In [21]:
# df_sirena=df_sirena[df_sirena['CLASS_NAME'].isin(['C','J','I','D','A'])]
# df_sirena.to_excel(r'123.xlsx',sheet_name='Flights',header=True, index=False,encoding='utf-8-sig')

In [22]:
%%time
allRBDs=df_sirena[['od','CLASS_NAME','numRBD']].sort_values(['numRBD']).drop_duplicates().groupby(['od'])['CLASS_NAME'].apply(lambda x: ''.join(x)).reset_index()


Wall time: 14 ms


In [23]:
%%time
#future flights
#get future flights for the next 90 days
query=f"""SELECT 
  lh.FLIGHT_NR,lh.DEP_DATE,lh.DEP_TIME, lh.ORIGIN,lh.DEST,
  lh.EQUIP_ID,
  s.RELEVANT_COMPETITOR,s.SCENARIO_NAME,
  ccd.MINUTES_BEFORE,ccd.MINUTES_AFTER,
  SUM(lh.LEG_BKD) "ACT_BKD",SUM(lh.LEG_GRP_BKD) "GRP_BKD", 
  SUM(lh.LEG_FRC_MIXED) "CALC_FRC_MIXED",--SUM(ls.CALC_FRC_FLOWN) "CALC_FRC_FLOWN",
  SUM(lh.LEG_CAP) "LEG_CAP", 
  (case when SUM(lh.LEG_CAP) <> 0 then round(SUM(lh.LEG_FRC_MIXED)/SUM(lh.LEG_CAP)*100,0) end) "FLF", 
  (case when SUM(lh.LEG_CAP) <> 0 then round((SUM(lh.LEG_BKD)+SUM(lh.LEG_GRP_BKD))/SUM(lh.LEG_CAP)*100,0) end) "BLF"
 FROM LEG_HEADER lh

LEFT JOIN SEGMENT_HEADER sh
  ON sh.FLIGHT_KEY=lh.FLIGHT_KEY AND sh.ORIGIN=lh.ORIGIN AND sh.DEST=lh.dest
LEFT JOIN (SELECT COMPCONN_KEY,MINUTES_BEFORE,MINUTES_AFTER from COMPETITOR_CONNECT_DETAILS 
            GROUP BY COMPCONN_KEY,MINUTES_BEFORE,MINUTES_AFTER) ccd
  ON ccd.COMPCONN_KEY=sh.COMPCONN_KEY
LEFT JOIN SCENARIOS s
  ON s.SCENARIO_ID=sh.SCENARIO_ID
  WHERE lh.DEP_DATE>= SYSDATE AND lh.DEP_DATE<=SYSDATE+90  AND lh.LAST_SNAP_NR IS NOT null

 -- lh.DEP_DATE=TO_DATE('2021/03/16', 'yyyy/mm/dd') and lh.FLIGHT_NR=267
  GROUP BY  
  
  lh.FLIGHT_NR,lh.DEP_DATE,lh.ORIGIN,lh.DEST,lh.DEP_TIME,
  lh.EQUIP_ID,
  s.RELEVANT_COMPETITOR,s.SCENARIO_NAME,
  ccd.MINUTES_BEFORE,ccd.MINUTES_AFTER"""

df_f = pd.read_sql(query, con=conn_rem)

Wall time: 23.4 s


In [24]:
# Convert time(1340 type str) to time(13:40:00 type datetime.time)
def convert_time(x):
    x = x.rjust(4,'0')
    x = x.ljust(6,'0')
    x = '{}:{}:{}'.format(x[:2],x[2:4],x[4:7])
    x = datetime.datetime.strptime(x, '%H:%M:%S').time()
    return x

# Convert time to count of minutes (13:40:00 to 820) from the beginnig of the day
def data_to_minutes_before(x):
    t = x.time()
    x = int(datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second).total_seconds())/60
    return x

# Convert time to count of minutes (1440 - (13:40:00(820)) from the end of the day
def data_to_minutes_after(x):
    t = x.time()
    x = 1440 - int(datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second).total_seconds())/60
    return x

# Creating to dates lower and higher some kind of range in which a competitor is considered a true competitor
def timedelata(x):
    date_delta = [x['flight_datetime'] - datetime.timedelta(minutes=x['MINUTES_BEFORE']),
                  x['flight_datetime'] + datetime.timedelta(minutes=x['MINUTES_AFTER'])]
    return date_delta

def min_price(x):
    o = x['o']
    d = x['d']
    y=df_Other[(df_Other['flight_datetime'] > x['delta_time'][0])  & 
                     (df_Other['flight_datetime'] < x['delta_time'][1])  &
                     (df_Other['o'] == o) & (df_Other['d'] == d)]
    if x['RELEVANT_COMPETITOR'] is None:
        return (y['price'].min())  
    else:
        return y[y['carrier'].isin(x['RELEVANT_COMPETITOR'].split(','))]['price'].min()

# # ДОДЕЛАТЬ!!!    
# # Func min_price with relevant direction
# def min_price(x):
#     o = x['o']
#     d = x['d']
#     rel_o = x['rel_o']
#     rel_d = x['rel_d']
# y=df_Other[(df_Other['flight_datetime'] > x['delta_time'][0])  & 
#                      (df_Other['flight_datetime'] < x['delta_time'][1])  &
#                      (df_Other['o'] == o) & (df_Other['d'] == d)]
#     if x['RELEVANT_COMPETITOR'] is None:
#         if x['RELEVANT_DIRECTION'] is None:
#             return (y['price'].min()) 
#         else:
#             y=df_Other[(df_Other['flight_datetime'] > x['delta_time'][0])  & 
#                      (df_Other['flight_datetime'] < x['delta_time'][1])  &
#                      (df_Other['o'] == rel_o) & (df_Other['d'] == rel_d)]
#             return(y['price'].min( ))
#     else:
#         return y[y['carrier'].isin(x['RELEVANT_COMPETITOR'].split(','))]['price'].min()

In [25]:
# df_Other[(df_Other['flight_datetime'] > '2021-06-28 00:00:00')  & 
#                      (df_Other['flight_datetime'] < '2021-06-29 00:00:00')  &
#                      (df_Other['o'] == 'VKO') & (df_Other['d'] =='ULV')]
# df_tmp[(df_tmp['o']=='VKO') & (df_tmp['d']=='ULV') & (df_tmp['DEP_DATE']=='2021-06-28')]

In [26]:
df_tmp=df_f
df_tmp['DEP_TIME'] = df_tmp['DEP_TIME'].astype(str).apply(convert_time)

In [27]:
# bringing to datetime
df_UT['flight_datetime'] = pd.to_datetime(df_UT['flight_datetime'])
df_UT['flight_date'] = pd.to_datetime(df_UT['flight_date'])
df_Other['flight_datetime'] = pd.to_datetime(df_Other['flight_datetime'])

# Flight_no to int
df_UT['flight_no'] = df_UT['flight_no'].astype(np.int64)

In [28]:
df_tmp = df_tmp.merge(df_UT, right_on=['flight_no', 'flight_date', 'o', 'd'],
                   left_on=['FLIGHT_NR','DEP_DATE','ORIGIN','DEST'] , how='left')

In [29]:
df_tmp=df_tmp[~df_tmp['flight_datetime'].isna()]
df_tmp=df_tmp[~((df_tmp['MINUTES_BEFORE'].isna() | df_tmp['MINUTES_BEFORE'].isna()) & (df_tmp['RELEVANT_COMPETITOR'].isna()))]
df_tmp=df_tmp.reset_index()

In [30]:
# Fill MINUTES_BEFORE and MINUTES_AFTER
df_tmp['MINUTES_BEFORE'].fillna(pd.Series(df_tmp.loc[df_tmp['MINUTES_BEFORE'].isna()]['flight_datetime'].apply(data_to_minutes_before)),inplace=True)
df_tmp['MINUTES_AFTER'].fillna(pd.Series(df_tmp.loc[df_tmp['MINUTES_AFTER'].isna()]['flight_datetime'].apply(data_to_minutes_after)),inplace=True)

In [31]:
%%time
df_tmp['delta_time'] = df_tmp.apply(timedelata, axis = 1)
df_tmp['min_price'] = df_tmp.apply(min_price,axis=1)

Wall time: 59.9 s


In [32]:
df_f=df_f.merge(df_tmp[['FLIGHT_NR','DEP_DATE','ORIGIN','DEST','min_price']], left_on=['FLIGHT_NR','DEP_DATE','ORIGIN','DEST'], right_on=['FLIGHT_NR','DEP_DATE','ORIGIN','DEST'],how='left')

In [33]:
%%time
#exclude charter and cancelled flights
#df_f=df_f[df_f['LAST_SNAP_NR'].notna()]
df_f=df_f[~df_f['FLIGHT_NR'].isin([401, 402, 403, 404, 405, 406, 407, 408, 409, 412, 413, 414, 415, 416, 417, 418, 419, 420,
 421, 422, 423, 424, 425, 429, 430, 431, 432, 677,678,679,680,1008,1009,1404,1405,1426,1427,1410,1411,1402,1403,426,427])]
df_f=df_f[df_f['FLIGHT_NR']<4000]
df_f=df_f[df_f['EQUIP_ID']!='AN4']


#future flights
#change df with future flights
dict_cap={'762':249,'735':116,'738':176,'734':150,"AT7":70,"AN4":48}
df_f['DAYS_TO_DEP']=((df_f['DEP_DATE']-yesterday)/np.timedelta64(1, 'D')).astype('int64')

#check digit +1
df_f['Sale_date']=df_f['DEP_DATE']-(df_f['DAYS_TO_DEP']+1).apply(lambda x: pd.Timedelta(x, unit='D'))
df_f['od']=df_f[['ORIGIN','DEST']].apply(lambda x: '-'.join(sorted(x)),axis=1)
df_f['last_cap']=df_f['EQUIP_ID'].map(dict_cap)
df_f['lf_by_last_cap']=df_f['ACT_BKD']/df_f['last_cap']
#df_f['flf']=df_f['frc']/df_f['last_cap']
df_f['month']=pd.DatetimeIndex(df_f['DEP_DATE']).month
df_f['year']=pd.DatetimeIndex(df_f['DEP_DATE']).year
df_f['ID_1242']=df_f[['od','month','year']].apply(lambda x: ''.join(x.values.astype(str)),axis=1)

Wall time: 525 ms


In [34]:
%%time
#curves for 2019 year by next 3 month

fields19=['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month']
df_comb2=pd.concat([df_f[fields19],df19[fields19]],ignore_index=True).drop_duplicates()
df_comb2=df_comb2.merge(df19[['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month','BLF']],left_on=fields19,right_on=fields19,how='left')
df_inter2=df_comb2
df_inter2=df_inter2.sort_values(by=['month','FLIGHT_NR','DAYS_TO_DEP'])
df_inter2=df_inter2.groupby(['FLIGHT_NR','month']).apply(lambda group:group.interpolate(method='linear'))
df_inter2=df_inter2.rename(columns={'BLF':'curveLF19'})

Wall time: 1.37 s


In [35]:
%%time
#goal LF for jun-aug for Boeings
goal_c=pd.read_excel(filepath+'rcurve19.xlsx', sheet_name='Лист8')
goal_curves=goal_c.melt(id_vars=["Месяц", "Вылет","Прилёт","Рейс","Тип ВС"], 
        var_name="SNAP_NR", 
        value_name="LF")
tmp_days_to_dep=df19[df19['FLIGHT_NR']==249][['SNAP_NR','DAYS_TO_DEP']].drop_duplicates()
goal_curves=goal_curves.merge(tmp_days_to_dep,left_on=['SNAP_NR'],right_on=['SNAP_NR'],how='left')
# goal_curves=goal_c.melt(id_vars=["Месяц", "Вылет","Прилёт","Рейс","Тип ВС"], 
#         var_name="SNAP_NR", 
#         value_name="LF")
goal_curves=goal_curves.rename(columns={'Месяц':'month','Вылет':'ORIGIN','Прилёт':'DEST','Рейс':'FLIGHT_NR'})

Wall time: 2min 15s


In [36]:
%%time
#curves for 2021 by next 3 month

fields21=['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month']
df_comb2=pd.concat([df_f[fields21],goal_curves[fields21]],ignore_index=True).drop_duplicates()
df_comb2=df_comb2.merge(goal_curves[['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month','LF']],left_on=fields21,right_on=fields21,how='left')
goal=df_comb2
goal=goal.sort_values(by=['month','FLIGHT_NR','DAYS_TO_DEP'])
goal=goal.groupby(['FLIGHT_NR','month']).apply(lambda group:group.interpolate(method='linear'))
goal=goal.rename(columns={'LF':'goalLF21'})
#df_inter2.to_excel(r'curve19.xlsx',header=True)

Wall time: 920 ms


In [37]:
%%time
#curves for 2019 year by next 3 month

fields19=['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month']
df_comb2=pd.concat([df_f[fields19],df19[fields19]],ignore_index=True).drop_duplicates()
df_comb2=df_comb2.merge(df19[['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','month','BLF']],left_on=fields19,right_on=fields19,how='left')
df_inter2=df_comb2
df_inter2=df_inter2.sort_values(by=['month','FLIGHT_NR','DAYS_TO_DEP'])
df_inter2=df_inter2.groupby(['FLIGHT_NR','month']).apply(lambda group:group.interpolate(method='linear'))
df_inter2=df_inter2.rename(columns={'BLF':'curveLF19'})

Wall time: 1.64 s


In [38]:
%%time
#combine two dataframes and interpolate booking_curves from snapshots to all days_to_dep 

fields=['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP']

df_c11=df_c.dropna()

df_comb=pd.concat([df_f[fields],df_c11[fields]],ignore_index=True).drop_duplicates()
df_comb=df_comb.merge(df_c11[['FLIGHT_NR', 'ORIGIN', 'DEST', 'DAYS_TO_DEP','lf_last_cap.mean','lf_last_cap.std']],left_on=fields,right_on=fields,how='left')

df_inter=df_comb
df_inter=df_inter.sort_values(by=['FLIGHT_NR','DAYS_TO_DEP'])
df_inter=df_inter.groupby('FLIGHT_NR').apply(lambda group:group.interpolate(method='linear'))

Wall time: 336 ms


In [39]:
%%time
#df_Infare=df_UT.merge(df_I[['price.min','price.mean','price.max','o','d','snapshot','flight_date']],left_on=['o','d','snapshot','flight_date'],right_on=['o','d','snapshot','flight_date'],how='left')
df_res=df_f.merge(df_inter[['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','lf_last_cap.mean','lf_last_cap.std']],left_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP'],right_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP'],how='left')

#add_goal_curves
df_res=df_res.merge(goal[['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month','goalLF21']],left_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month'],right_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month'],how='left')
df_res=df_res.drop_duplicates()
df_res['goalLF21']=df_res['goalLF21']/100
#add Infare_FG
df_res=df_res.merge(df_InfareFG[['snapshot','flight_date','flight_no','o','d','UT_price_infare','min', 'mean',
       'max']],left_on=['DEP_DATE','FLIGHT_NR','ORIGIN','DEST'],right_on=['flight_date','flight_no','o','d'],how='left')



Wall time: 186 ms


In [40]:
%%time
#fill curves of new flights to them route curves
df_res['lf_last_cap.mean'] =  df_res['lf_last_cap.mean'].fillna(df_res.groupby(['ORIGIN','DEST','DAYS_TO_DEP'])['lf_last_cap.mean'].transform('median'))
df_res['lf_last_cap.std'] =  df_res['lf_last_cap.std'].fillna(df_res.groupby(['ORIGIN','DEST','DAYS_TO_DEP'])['lf_last_cap.std'].transform('median'))


Wall time: 23 ms


In [41]:
%%time
#fill curves of new routes to 2019 year curves
df_res=df_res.merge(df_inter2[['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month','curveLF19']],
                    left_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month'],right_on=['FLIGHT_NR','ORIGIN','DEST','DAYS_TO_DEP','month'],how='left')
df_res['lf_last_cap.mean']=df_res['lf_last_cap.mean'].fillna(df_res['curveLF19'])

Wall time: 52 ms


In [42]:
#take min between rembarndt lf and lf by lasct capacity
df_res['BLF']=df_res['BLF']/100
df_res['LF']=df_res[['lf_by_last_cap','BLF']].min(axis=1)
df_res['BLF']=df_res['BLF']*100

In [43]:
%%time
#booking curve for the network divides by the Moscow network curve and Regional network curve


df_cMOW=df_inter[((df_inter['ORIGIN']=='VKO')|(df_inter['DEST']=='VKO'))].groupby(['DAYS_TO_DEP']).agg({'lf_last_cap.mean':'median','lf_last_cap.std':'median'}).reset_index()
df_cREG=df_inter[~((df_inter['ORIGIN']=='VKO')|(df_inter['DEST']=='VKO'))].groupby(['DAYS_TO_DEP']).agg({'lf_last_cap.mean':'median','lf_last_cap.std':'median'}).reset_index()


df_cMOW=df_cMOW.rename(columns={'lf_last_cap.mean':'MOW network mean LF','lf_last_cap.std':'MOW network std dev'})
df_cREG=df_cREG.rename(columns={'lf_last_cap.mean':'REG network mean LF','lf_last_cap.std':'REG network std dev'})

Wall time: 25 ms


In [44]:
df_res=df_res.merge(df_cMOW[['DAYS_TO_DEP','MOW network mean LF','MOW network std dev']],
                    left_on=['DAYS_TO_DEP'],
                    right_on=['DAYS_TO_DEP'],how='left')

In [45]:
df_res=df_res.merge(df_cREG[['DAYS_TO_DEP','REG network mean LF','REG network std dev']],
                    left_on=['DAYS_TO_DEP'],
                    right_on=['DAYS_TO_DEP'],how='left')

In [46]:
m1 = df_res['od'].str.contains('VKO')
m2= ~df_res['od'].str.contains('VKO')

df_res.loc[m1,'lf_last_cap.mean']=df_res.loc[m1,'lf_last_cap.mean'].fillna(df_res['MOW network mean LF'])
df_res.loc[m1,'lf_last_cap.std']=df_res.loc[m1,'lf_last_cap.std'].fillna(df_res['MOW network std dev'])

df_res.loc[m2,'lf_last_cap.mean']=df_res.loc[m2,'lf_last_cap.mean'].fillna(df_res['REG network mean LF'])
df_res.loc[m2,'lf_last_cap.std']=df_res.loc[m2,'lf_last_cap.std'].fillna(df_res['REG network std dev'])

df_res['goalLF21']=df_res['goalLF21'].fillna(df_res['curveLF19'])
df_res['goalLF21']=df_res['goalLF21'].fillna(df_res['lf_last_cap.mean'])

df_res['act_lf-curve']=df_res['LF']-df_res['goalLF21']

In [47]:
%%time
filepath = '//utair.dom/ut/UTPA_OUD/Запорощенко/'
#Controllers=pd.read_excel(filepath+'Controllers.xlsx',sep=',')
Controllers=pd.read_excel(filepath+'Controllers.xlsx')
df_res=df_res.merge(Controllers[['citypair','controller']], left_on='od',right_on='citypair',how='left')


Wall time: 1.03 s


In [48]:
%%time
#rename and delete excess columns
useless=['Sale_date','ID_1242','citypair',
         'flight_date','flight_no', 'o', 'd',
         'ACT_BKD','GRP_BKD', 'CALC_FRC_MIXED']
df_res=df_res.drop(columns=useless)
df_res=df_res.rename(columns={'last_cap':'Ёмкость','CLASS_NAME':'RBD','lf_last_cap.mean':'Кривая LF','ORIGIN':'O','DEST':'D','FLIGHT_NR':'FL_NR'})


df_res=df_res.rename(columns={'FL_NR':'Рейс','DEP_DATE':'Дата вылета',
                              'O':'Вылет','D':'Прилет',
                              'EQUIP_ID':'Тип ВС','DAYS_TO_DEP':'Дней до вылета',
                             #'lf_by_last_cap':'LF',
                              'goalLF21':'mean LF',#Кривая LF':'mean LF',
                              'lf_last_cap.std':'std_dev','act_lf-curve':'Откл. От среднего'})

Wall time: 28.1 ms


In [49]:
%%time
# df_rcnt['date']=df_rcnt['date'].apply(convert_datetime)

dict_RBD={
    'F':26,'Z':25, 'P':24,  'K':23,  'L':22,  'H':21,  'V':20,  'R':19,  'O':18,  'U':17, 'W':16,'X':15,'B':14,
    'N':13,  'G':12,  'Q':11,  'E':10,  'T':9,  'S':8,  'Y':7,  'A':5,  'D':4,  'I':3,'J':2,'C':1
}

Y_RBD=['F','Z','P','K','L','H','V','O','U','W','X','B','N','E','T','S','Y']

df_rbd['numRBD']=df_rbd['rbd'].map(dict_RBD)
dfY=df_rbd[df_rbd['cabin']=='Y']
dfY['OD']=dfY[['departureAirport','arrivalAirport']].apply(lambda x: '-'.join(sorted(x)),axis=1)
dfY=dfY.merge(allRBDs,left_on=['OD'],right_on=['od'],how='left')

dfY['CLASS_NAME']=dfY['CLASS_NAME'].fillna('')
dfY['actual rbd']=dfY.apply(lambda x: x.rbd in x.CLASS_NAME, axis=1)

min_rbd=dfY[(dfY['state']=='open') & (dfY['rbd_state']=='open') &(dfY['rbd_av']>0) & (dfY['actual rbd']==True)].sort_values(by='numRBD',ascending=False).groupby(['number','date','aircraft','departureAirport','arrivalAirport']).first().reset_index()
min_rbd['updated2']=pd.to_datetime(min_rbd.updated).dt.tz_localize(None)


df_res['Дата вылета']= df_res['Дата вылета'].apply(convert_datetime)
min_rbd['date']= min_rbd['date'].apply(convert_datetime)


<timed exec>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 11.7 s


In [50]:
%%time
df_rbd2=df_rbd.merge(df_sirena[['ORIGIN','DEST','CLASS_NAME','UT_price']],left_on=['departureAirport','arrivalAirport','rbd'],right_on=['ORIGIN','DEST','CLASS_NAME'],how='left')

Wall time: 685 ms


In [51]:
df_rbd.columns

Index(['number', 'date', 'aircraft', 'departureAirport', 'arrivalAirport',
       'legId', 'available', 'booked', 'updated', 'state', 'cabin', 'ca_av',
       'ca_bkd', 'ca_wait', 'rbd', 'rbd_state', 'rbd_av', 'rbd_bkd',
       'rbd_wait', 'numRBD'],
      dtype='object')

In [52]:
# df_rbd=df_rbd.sort_values(by=['number','date','departureAirport','arrivalAirport','numRBD'])

# df_rbd=df_rbd[df_rbd['rbd_state']=='open']
# df_rbd['rbd_av-1'] = df_rbd.groupby(['number','date','departureAirport','arrivalAirport'])['rbd_av'].shift(periods=-1)
# df_rbd['rbd_av-1']=df_rbd['rbd_av-1'].fillna(0)
# df_rbd['quote_AV']=df_rbd['rbd_av']-df_rbd['rbd_av-1']

In [53]:
#df_rbd[(df_rbd['date']==datetime.date(2021, 4, 13)) & (df_rbd['number']==139)]

In [54]:
%%time
#df_rbd.date.unique()
#df_rbd[(df_rbd.number==329) & (df_rbd.date==datetime.date(2021, 3, 18))]

bkd=pd.pivot_table(df_rbd,values=['rbd_bkd'],index=['number','date','departureAirport','arrivalAirport'],columns=['numRBD','rbd']).reset_index()
bkd=bkd.fillna('')
bkd=bkd.sort_index(axis=1,ascending=False)
#bkd.columns=bkd.columns.droplevel([0,1])
bkd.columns=['F', 'Z', 'P', 'K', 'L', 'H', 'V', 'R', 'O', 'U', 'W', 'X', 'B', 'N',
       'G', 'Q', 'E', 'T', 'S', 'Y', 'A', 'D', 'I', 'J', 'C', 'Num', 'Origin', 'Date', 'Dest']
bkd['Date']= bkd['Date'].apply(convert_datetime)

Wall time: 1.37 s


In [55]:
# #df_rbd.date.unique()
# #df_rbd[(df_rbd.number==329) & (df_rbd.date==datetime.date(2021, 3, 18))]

# av=pd.pivot_table(df_rbd,values=['rbd_av'],index=['number','date','departureAirport','arrivalAirport'],columns=['numRBD','rbd']).reset_index()
# av=av.fillna('')
# av=av.sort_index(axis=1,ascending=False)
# #bkd.columns=bkd.columns.droplevel([0,1])
# av.columns=['F', 'Z', 'P', 'K', 'L', 'H', 'V', 'R', 'O', 'U', 'W', 'X', 'B', 'N',
#        'G', 'Q', 'E', 'T', 'S', 'Y', 'A', 'D', 'I', 'J', 'C', 'Num', 'Origin', 'Date', 'Dest']
# av['Date']= av['Date'].apply(convert_datetime)

In [56]:
# av.Date.unique()
# av[(av['Date']=='10.04.2021') & (av['Num']==573)].transpose()

In [57]:
%%time
df_res2=df_res.merge(min_rbd[['number', 'date', 'departureAirport', 'arrivalAirport',
       'available', 'booked', 'updated2', 'rbd', 'rbd_av', 'rbd_bkd','numRBD']],
                     left_on=['Рейс','Дата вылета','Вылет','Прилет'],
                     right_on=['number','date','departureAirport','arrivalAirport'],how='left')

df_res2=df_res2.merge(allRBDs[['od','CLASS_NAME']],left_on=['od'],right_on=['od'],how='left')

# df_res2=df_res2.merge(df_rcnt[['date','number','rcnt']],
#                       left_on=['Дата вылета','Рейс'],
#                       right_on=['date','number'],how='left')

df_res2['CLASS_NAME']=df_res2['CLASS_NAME'].fillna('')

df_res2['minAVRBD']=df_res2['CLASS_NAME'].astype(str).apply(lambda x: x[-1:])
df_res2['rbd']=df_res2['rbd'].astype(str)
#df_res2['rbd']= df_res2.apply(lambda row: row.rbd if ((row.rbd in row.CLASS_NAME)|(row.rbd=='nan')) else row.CLASS_NAME[-1:], axis=1)

df_res2['min2']=df_res2[['min','mean','max']].min(axis=1)
df_res2['mean2']=df_res2[['min','mean','max']].median(axis=1)
df_res2['max2']=df_res2[['min','mean','max']].max(axis=1)

df_res2=df_res2.merge(bkd,left_on=['Рейс', 'Дата вылета','Вылет', 'Прилет'],right_on=['Num', 'Date','Origin', 'Dest'],how='left')

# Drop tax and UT_price from merge
df_res3=df_res2.merge(df_sirena[['ORIGIN', 'DEST', 'CLASS_NAME','type','FARE','sum_manual', 'numRBD', 'od',
       'fare-3', 'fare-2','fare-1', 'fare+1', 'fare+2','fare+3','DEP_DATE_FROM','DEP_DATE_UNTIL']],
                      left_on=['Вылет', 'Прилет','rbd'],right_on=['ORIGIN', 'DEST', 'CLASS_NAME'],
                      how='left').rename(columns={'sum_manual':'YQ_tax'})

df_res3['DEP_TIME'] = df_res3['DEP_TIME'].astype(str)
df_res3['flight_datetime'] = df_res3[['Дата вылета','DEP_TIME']].apply(lambda x: parser.parse(x[0]+" "+x[1],dayfirst=True), axis=1)

# Add field infare_tax_y to df_res3
df_res3=df_res3.merge(tax_inf[['flight_datetime','origin','destination','flight_no','infare_tax_y']],
                        left_on=['flight_datetime','Вылет','Прилет','Рейс'],
                        right_on=['flight_datetime','origin','destination','flight_no'],how = 'left').rename(columns={'infare_tax_y':'Infare_tax'})



Wall time: 2.05 s


# Вставка 26.05.2021

In [58]:
# def set_tax(x):
#     o = x['Вылет']
#     d = x['Прилет']
#     f_n = x['Рейс']
#     if np.isnan(x['infare_tax_y']):
#         tax = mean_tax.loc[(mean_tax['origin'] == o)  &
#                            (mean_tax['destination'] == d) & (mean_tax['flight_no'] == f_n)]['infare_tax_y'].min()
#         return tax
#     else:
#         return x['infare_tax_y']

# df_res3_2['infare_tax_y'] = df_res3_2.apply(set_tax,axis=1)

In [59]:
def addition_tax1(x):
    if not np.isnan(x['Infare_tax']):
        return 'Infare_tax'
    else:
        if not np.isnan(x['YQ_tax']):
            return 'YQ_tax'
        else:
            return np.nan
        
def addition_tax2(x):
    if not np.isnan(x['Infare_tax']):
        return x['Infare_tax']
    else:
        if not np.isnan(x['YQ_tax']):
            return x['YQ_tax']
        else:
            return np.nan

    

In [60]:
%%time
# Fill infare_tax mean on each flight
df_res3['Infare_tax'] = df_res3['Infare_tax'].fillna(df_res3.groupby(by=['Вылет','Прилет','Рейс'])['Infare_tax'].transform('mean'))

# The field shows from where the collection will be pulled
df_res3['Source_tax'] = df_res3.apply(addition_tax1,axis=1)

# Take Infare_tax in priority
df_res3['General_tax'] = df_res3.apply(addition_tax2,axis=1)

# df_res3['infare_tax_y'] = df_res3['infare_tax_y'].fillna(df_res3['YQ_tax'])

Wall time: 1.1 s


In [61]:
df_res3['UT_price']=df_res3.apply(lambda x: x['FARE'] if np.isnan(x['General_tax']) else x['General_tax']+x['FARE'],axis=1)
df_res3['tax']=df_res3.apply(lambda x: 'fare' if ((np.isnan(x['General_tax'])) | (x['General_tax'] == 0)) else 'tax+fare',axis=1)


# Вставка

In [62]:
# Return 1 if tariff is actual else return 0
def tariff_actual(x):
    if x['DEP_DATE_FROM'] is not pd.NaT:
        if x['DEP_DATE_UNTIL'] is not pd.NaT:
            if x['DEP_DATE_FROM'] <= parser.parse(x['Дата вылета'], dayfirst = True) < x['DEP_DATE_UNTIL']:
                return 1
            else:
                return 0
        else:
            if x['DEP_DATE_FROM'] <= parser.parse(x['Дата вылета'], dayfirst = True) < parser.parse('01.11.2500'):
                return 1
            else:
                return 0
    else:
        return 1
    
    
df_res3['tariff_actual'] = df_res3.apply(tariff_actual,axis = 1)
df_res3.drop(df_res3[df_res3['tariff_actual'] == 0].index, inplace=True)
df_res3.drop(['DEP_DATE_FROM','DEP_DATE_UNTIL','tariff_actual'],axis=1,inplace=True)

In [63]:
# Convert time into more comfortable
df_res3['Дата вылета'] = df_res3['Дата вылета'].apply(lambda x: parser.parse(x, dayfirst=True).date())
df_sales['depdate'] = df_sales['depdate'].apply(lambda x: parser.parse(x, dayfirst=True).date()) 

In [64]:
# Create dict_FARE {'AAQ-AER': 7000, 1500, 2500, 5000, 2000} etc.
dict_FARE = df_sirena.groupby('od').FARE.apply(set).apply(list).to_dict()

# Sort and convert into nparray {'AAQ-AER': array([1000., 1500., 2500., 5000., 7000.])} etc.
for i in dict_FARE:
    dict_FARE[i].sort()
    dict_FARE[i] = np.asarray(dict_FARE[i])

In [65]:
def min_max_FARE(x):
    try:
        if dict_FARE[x['od_x']][0] == x['FARE']:
            return 'Минимальный'
        if dict_FARE[x['od_x']][-1] == x['FARE']:
            return 'Максимальный'
        else:
            return np.nan
    except KeyError:
        return "Маршрута нет в сирене"

In [66]:
df_res3['Мин/Макс тариф?'] = df_res3[['od_x','FARE']].apply(min_max_FARE,axis=1)

In [67]:
%%time
df_res3=df_res3.merge(df_sales[['fl_num', 'Пункт вылета', 'Пункт прилета', 'rev_tct', 'Pax', 'avg_fare', 'depdate']],left_on=['Рейс','Дата вылета','Вылет','Прилет'],right_on=['fl_num','depdate','Пункт вылета','Пункт прилета'],how='left')

Wall time: 447 ms


# Add prices Rcnt bookings and merge with res3

In [68]:
mean_tax=tax_inf.groupby(['origin','destination']).agg({'infare_tax_y':'mean','YQ_tax':'mean'}).reset_index()

In [69]:
mean_tax['compare']=mean_tax['infare_tax_y']==mean_tax['YQ_tax']

In [71]:
# df_rcnt_tmp=df_rcnt_with_code1.merge(df_sirena[['FARE','ORIGIN', 'DEST','CLASS_NAME','DEP_DATE_FROM','DEP_DATE_UNTIL']],left_on=['departureAirport','arrivalAirport','last_code'],
#                                           right_on=['ORIGIN', 'DEST','CLASS_NAME'],how='left')

In [72]:
# # Rename feald date and convert datetime
# df_rcnt_tmp=df_rcnt_tmp.rename(columns={'date':'Дата вылета'})
# df_rcnt_tmp['Дата вылета']= df_rcnt_tmp['Дата вылета'].apply(convert_datetime)

# # leaves only the current tariffs
# df_rcnt_tmp['tariff_actual'] = df_rcnt_tmp.apply(tariff_actual,axis = 1)
# df_rcnt_tmp.drop(df_rcnt_tmp[df_rcnt_tmp['tariff_actual'] == 0].index, inplace=True)
# df_rcnt_tmp.drop(['DEP_DATE_FROM','DEP_DATE_UNTIL','tariff_actual'],axis=1,inplace=True)

# # Get field infare_tax from mean_tax df
# df_rcnt_tmp=df_rcnt_tmp.merge(mean_tax[['infare_tax_y','origin','destination']],left_on=['departureAirport','arrivalAirport'],
#                                           right_on=['origin','destination'],how='left')

# df_rcnt_tmp['rcnt_price']=df_rcnt_tmp['FARE']+df_rcnt_tmp['infare_tax_y']
# df_rcnt_tmp['rcnt_rev']=df_rcnt_tmp['rcnt_price']*df_rcnt_tmp['rcnt']

# df_rcnt_tmp = df_rcnt_tmp[~(df_rcnt_tmp['last_code'] == 'G')]
# df_rcnt_tmp = df_rcnt_tmp[(df_rcnt_tmp['rcnt'] > 0) & (~df_rcnt_tmp['FARE'].isna())]

In [73]:
# df_rcnt=df_rcnt_tmp.groupby(['Дата вылета','number','origin','destination']).agg({'rcnt':'sum','rcnt_rev':'sum'}).reset_index()

# df_rcnt['rcnt_avg_fare']=round(np.divide(df_rcnt['rcnt_rev'],df_rcnt['rcnt']),0)

# df_rcnt['rcnt_avg_fare']=df_rcnt[['rcnt','rcnt_rev','rcnt_avg_fare']].apply(lambda x: x['rcnt_rev']
#                                                                             if ((x['rcnt_avg_fare'] == np.inf) | (x['rcnt_avg_fare'] == -np.inf)) 
#                                                                             else x['rcnt_avg_fare'],axis=1)

# df_rcnt['rcnt_avg_fare'] = df_rcnt['rcnt_avg_fare'].fillna(0)
# df_rcnt['Дата вылета'] = df_rcnt['Дата вылета'].apply(lambda x: parser.parse(x, dayfirst=True).date())



In [77]:
# Add fealds rcnt and rcnt_avg_fare to main df df_res3
# df_res3=df_res3.merge(df_rcnt[['Дата вылета','number','origin','destination','rcnt',
#                                #'rcnt_avg_fare'
#                               ]],
#                       left_on=['Дата вылета','Рейс','Вылет','Прилет'],
#                       right_on=['Дата вылета','number','origin','destination'],how='left')

# End add prices Rcnt bookings and merge with res3

In [78]:
%%time
#загрузка ниже кривой-станд.откл. - 1, выше - 0, -1 - если есть NA
df_res3['LF<mean-std']=np.where(df_res3['LF'].isnull(),
                               -1,
                               (df_res3['LF']<(df_res3['mean LF']-df_res3['std_dev']))) 

#загрузка выше кривой+станд.откл. - 1, ниже - 0, -1 - если есть NA
df_res3['LF>mean+std']=np.where(df_res3['LF'].isnull(),
                               -1,
                               (df_res3['LF']>(df_res3['mean LF']+df_res3['std_dev'])))
#тариф UT ниже минимального тарифа конкурентов - 1, выше - 0, -1 - если отсутствует одно из значений
df_res3['UT<min']=np.where((df_res3['UT_price'].isnull()) | (df_res3['min2'].isnull()),
                               -1,
                               (df_res3['UT_price']<df_res3['min2']))
#тариф UT ниже среднего из 3 тарифов конкурентов - 1, выше - 0, -1 - если отсутствует одно из значений
df_res3['UT<mean']=np.where((df_res3['UT_price'].isnull())  | (df_res3['mean2'].isnull()),
                               -1,
                               (df_res3['UT_price']<df_res3['mean2']))
#тариф UT ниже максимального из 3 тарифов конкурентов - 1, выше - 0, -1 - если отсутствует одно из значений
df_res3['UT<max']=np.where((df_res3['UT_price'].isnull()) | (df_res3['max2'].isnull()),
                               -1,
                               (df_res3['UT_price']<df_res3['max2']))


Wall time: 7.47 ms


In [79]:
%%time
df_depLF=df[df['SNAP_NR']==17].groupby('FLIGHT_NR').agg({'BLF':'mean'}).reset_index()
df_depLF=df_depLF.rename(columns={'BLF':'depLF'})
df_depLF['depLF']=df_depLF['depLF'].fillna(0)
df_depLF['depLF']=df_depLF['depLF'].astype(int)

df_res3=df_res3.merge(df_depLF,left_on='Рейс',right_on='FLIGHT_NR',how='left')

Wall time: 86.5 ms


In [80]:
df_res3.columns.tolist()

['Рейс',
 'Дата вылета',
 'DEP_TIME',
 'Вылет',
 'Прилет',
 'Тип ВС',
 'RELEVANT_COMPETITOR',
 'SCENARIO_NAME',
 'MINUTES_BEFORE',
 'MINUTES_AFTER',
 'LEG_CAP',
 'FLF',
 'BLF',
 'min_price',
 'Дней до вылета',
 'od_x',
 'Ёмкость',
 'lf_by_last_cap',
 'month',
 'year',
 'Кривая LF',
 'std_dev',
 'mean LF',
 'snapshot',
 'UT_price_infare',
 'min',
 'mean',
 'max',
 'curveLF19',
 'LF',
 'MOW network mean LF',
 'MOW network std dev',
 'REG network mean LF',
 'REG network std dev',
 'Откл. От среднего',
 'controller',
 'number',
 'date',
 'departureAirport',
 'arrivalAirport',
 'available',
 'booked',
 'updated2',
 'rbd',
 'rbd_av',
 'rbd_bkd',
 'numRBD_x',
 'CLASS_NAME_x',
 'minAVRBD',
 'min2',
 'mean2',
 'max2',
 'F',
 'Z',
 'P',
 'K',
 'L',
 'H',
 'V',
 'R',
 'O',
 'U',
 'W',
 'X',
 'B',
 'N',
 'G',
 'Q',
 'E',
 'T',
 'S',
 'Y',
 'A',
 'D',
 'I',
 'J',
 'C',
 'Num',
 'Origin',
 'Date',
 'Dest',
 'ORIGIN',
 'DEST',
 'CLASS_NAME_y',
 'type',
 'FARE',
 'YQ_tax',
 'numRBD_y',
 'od_y',
 'fare

In [85]:
%%time
#cписок рейсов с низкой кривой
lowLF=['OMS-SGC','KUF-TJM','UFA-VOG','NJC-OMS','AER-GRV',
'NUX-TJM','NJC-TJM','ASF-MRV','PEE-UFA',
'CEK-HMA','KUF-SGC','NUX-UFA','HMA-SGC',
'KZN-LPK','KUF-NJC','ASF-VOG','KJA-TJM',
'ESL-MRV','KUF-SGC','KUF-NJC']#'SCW-VKO','LPK-MRV'

f1=-400 #Тариф ниже средней группы конкурентов на f1 рублей
f2=300 #Тариф выше мин.конкурента на f2 рублей
f3=500# Если мы выше максимального конкурента - 500 рублей
#категории отклонения по загрузке
d1=0.3
d2=0.1
d3=-0.3
d4=-0.1

d5=30
d6=10
                            
                              
                              
# df_res3['% rcnt']=round(np.divide(df_res3['rcnt'], df_res3['Ёмкость'])*100,2)                              
df_res3['Изменение тарифа']=np.where(((df_res3['LF>mean+std']==1)&(df_res3['mean2']>0)),
df_res3['UT_price']-df_res3['mean2'],
                            np.where(((df_res3['LF<mean-std']==1)&(df_res3['min2']>0)),
df_res3['UT_price']-df_res3['min2'],0))



df_res3['категория откл']=np.where((df_res3['Откл. От среднего']>d1),
"{:.0f}%".format(round(d1*100, 0)),
                          np.where((df_res3['Откл. От среднего']>d2),
"{:.0f}%".format(round(d2*100, 0)),
                          np.where((df_res3['Откл. От среднего']<d3),
"{:.0f}%".format(round(d3*100, 0)),
                          np.where((df_res3['Откл. От среднего']<d4),
"{:.0f}%".format(round(d4*100, 0)),''))))

#df_res3['Изменение тарифа']=df_res3['Изменение тарифа'].astype(int)  

df_res3['Категория']=np.where((((df_res3['available']==0)|(df_res3['available'].isna())) & (df_res3['LF']>0.9)),
                            'Нет мест для продажи',
np.where((((df_res3['available']==0)|(df_res3['available'].isna()))),
    'Продажа закрыта',
            np.where(((df_res3['min_price']>0) &(df_res3['LF>mean+std']==1) & (df_res3['категория откл']!='')&
                      ((df_res3['UT_price']-df_res3['min_price'])<-1200)),
                        'Загрузка высокая, тариф низкий',
                         np.where(((df_res3['min_price']>0)&(df_res3['LF<mean-std']==1) &(df_res3['категория откл']!='')&
                                  (df_res3['fare-1']!='min')&((df_res3['UT_price']-df_res3['min_price'])>f2)),        
                                'Загрузка низкая, тариф высокий',
                                  
        np.where(((df_res3['LF>mean+std']==1)&(df_res3['Изменение тарифа']<f1)&
        (df_res3['категория откл']!='')&(df_res3['UT_price']<df_res3['avg_fare']*2)
        &(((df_res3['UT_price']<df_res3['max2']*0.8)&(df_res3['UT_price']<df_res3['max2']*0.8)
                                ))),
'Загрузка высокая, тариф низкий',
                              
                              
                             
#если  отстаем от кривой и выше конкурента >200 и наш тариф не минимальный, то загрузка низкая, тариф высокий
                     np.where(((df_res3['LF<mean-std']==1)&(df_res3['Изменение тарифа']>f2)&
                               (df_res3['fare-1']!='min')&(df_res3['категория откл']!='')),         
'Загрузка низкая, тариф высокий',
#если нет конкурентов и отстаем от кривой и наш тариф не минимальный, то загрузка низкая, тариф высокий
                    np.where(((df_res3['min_price']==0) &(df_res3['LF<mean-std']==1)& (df_res3['UT<min']==-1)&
                               (df_res3['fare-1']!='min')&(df_res3['категория откл']!='')),         
'Загрузка низкая, нет конкурентов',   
                     np.where(((df_res3['% rcnt']>d6)&(df_res3['Откл. От среднего']>d2)&(df_res3['min2'].isna())&(df_res3['Дней до вылета']>6)),
'Rcnt bkd >10% capacity',
                    #если выше кривой+стд откл и нет конкурентов и наш тариф не максимальный, то Загрузка высокая, нет конкурентов
                    np.where(((df_res3['min_price']==0) &(df_res3['LF>mean+std']==1)&(df_res3['UT<mean']==-1)&
                               (df_res3['fare+1']!='max')&(df_res3['категория откл']!='')&(df_res3['avg_fare']>df_res3['UT_price']) ),
'Загрузка высокая, нет конкурентов',
                     np.where((
                         (df_res3['od_x'].isin(lowLF)) & (df_res3['Откл. От среднего']>d4)&
                               (df_res3['fare-1']!='min')&(df_res3['Откл. От среднего']<=-0.05) &(df_res3['UT<min']==-1)),         
'Низкая загрузка',  
                     np.where((
                         (~df_res3['max2'].isna())&
                         (df_res3['UT_price']>(df_res3['max2']-500)) &
                         (df_res3['% rcnt']<=0)&
                         (df_res3['available']>15)),         
'Высокий тариф, нет продаж',   
                     np.where(((df_res3['Откл. От среднего']>=0)&(df_res3['UT_price']<df_res3['min'])),         
'Тариф ниже мин.группы конкурентов',
                     np.where(((df_res3['available']<10)&(df_res3['numRBD_x']>13)&(df_res3['Дней до вылета']>2)),         
'Меньше 10 свободных мест и тариф ниже N',
#                     np.where((df_res3['Откл. От среднего']>d1),         
#'Высокое откл. от кривой >'+"{:.2f}%".format(round(d1*100, 2)),
#                     np.where((df_res3['Откл. От среднего']<d3),         
#'Высокое откл. от кривой <'+"{:.2f}%".format(round(d3*100, 2)),
                     np.where(((df_res3['fare-1']=='min')&(df_res3['Откл. От среднего']>0.2)&(df_res3['CLASS_NAME_x'].str.len()>1)),
'Открыт min RBD, откл от кривой >20%',
                     np.where(((df_res3['fare+1']=='max')&(df_res3['numRBD_x']!=13)&((df_res3['CLASS_NAME_x'].str.len()>1))),
'Открыт max RBD',
                     np.where(((df_res3['Откл. От среднего']>=-0.15)&((np.where(((df_res3['fare+1']=='max')|(df_res3['fare+1'].isna())),df_res3['UT_price'],df_res3['fare+1'])+df_res3['UT_price'])<df_res3['min'])),
'Следующий RBD ниже мин.конкурента',
'Норма'))))))))))))))))


                                   

df_res3['Москва/Регионы']=np.where((df_res3['od_x'].str.contains('VKO')),'MOW','REG')
df_res3['Комментарий']=""

KeyError: 'Откл. От среднего'

In [82]:
# %%time
# #cписок рейсов с низкой кривой
# lowLF=['OMS-SGC','KUF-TJM','UFA-VOG','NJC-OMS','AER-GRV',
# 'NUX-TJM','NJC-TJM','ASF-MRV','PEE-UFA',
# 'CEK-HMA','KUF-SGC','NUX-UFA','HMA-SGC',
# 'KZN-LPK','KUF-NJC','ASF-VOG','KJA-TJM',
# 'ESL-MRV','KUF-SGC','KUF-NJC']#'SCW-VKO','LPK-MRV'

# f1=-400 #Тариф ниже средней группы конкурентов на f1 рублей
# f2=300 #Тариф выше мин.конкурента на f2 рублей
# f3=500# Если мы выше максимального конкурента - 500 рублей
# #категории отклонения по загрузке
# d1=0.3
# d2=0.1
# d3=-0.3
# d4=-0.1

# d5=30
# d6=10  

# df_res3['Категория2']=np.where((((df_res3['available']==0)|(df_res3['available'].isna())) & (df_res3['LF']>0.9)),
# 'Нет мест для продажи',
#                      np.where((((df_res3['available']==0)|(df_res3['available'].isna()))),
# 'Продажа закрыта',
                              
#                      np.where(((df_res3['LF>mean+std']==1)&(df_res3['Изменение тарифа']<f1)&
#                                (df_res3['категория откл']!='')&(df_res3['UT_price']<df_res3['avg_fare']*2)
#                                &(((df_res3['UT_price']<df_res3['max2']*0.8)&(df_res3['UT_price']<df_res3['max2']*0.8)
#                                 ))),
# 'Загрузка высокая, тариф низкий',
                              
                              
                             
# #если  отстаем от кривой и выше конкурента >200 и наш тариф не минимальный, то загрузка низкая, тариф высокий
#                      np.where(((df_res3['LF<mean-std']==1)&(df_res3['Изменение тарифа']>f2)&
#                                (df_res3['fare-1']!='min')&(df_res3['категория откл']!='')),         
# 'Загрузка низкая, тариф высокий',
# #если нет конкурентов и отстаем от кривой и наш тариф не минимальный, то загрузка низкая, тариф высокий
#                     np.where(((df_res3['LF<mean-std']==1)& (df_res3['UT<min']==-1)&
#                                (df_res3['fare-1']!='min')&(df_res3['категория откл']!='')),         
# 'Загрузка низкая, нет конкурентов',   
#                      np.where(((df_res3['% rcnt']>d6)&(df_res3['Откл. От среднего']>d2)&(df_res3['min2'].isna())&(df_res3['Дней до вылета']>6)),
# 'Rcnt bkd >10% capacity',
#                     #если выше кривой+стд откл и нет конкурентов и наш тариф не максимальный, то Загрузка высокая, нет конкурентов
#                     np.where(((df_res3['LF>mean+std']==1)&(df_res3['UT<mean']==-1)&
#                                (df_res3['fare+1']!='max')&(df_res3['категория откл']!='')&(df_res3['avg_fare']>df_res3['UT_price']) ),
# 'Загрузка высокая, нет конкурентов',
#                      np.where((
#                          (df_res3['od_x'].isin(lowLF)) & (df_res3['Откл. От среднего']>d4)&
#                                (df_res3['fare-1']!='min')&(df_res3['Откл. От среднего']<=-0.05) &(df_res3['UT<min']==-1)),         
# 'Низкая загрузка',  
#                      np.where((
#                          (~df_res3['max2'].isna())&
#                          (df_res3['UT_price']>(df_res3['max2']-500)) &
#                          (df_res3['% rcnt']<=0)&
#                          (df_res3['available']>15)),         
# 'Высокий тариф, нет продаж',   
#                      np.where(((df_res3['Откл. От среднего']>=0)&(df_res3['UT_price']<df_res3['min'])),         
# 'Тариф ниже мин.группы конкурентов',
#                      np.where(((df_res3['available']<10)&(df_res3['numRBD_x']>13)&(df_res3['Дней до вылета']>2)),         
# 'Меньше 10 свободных мест и тариф ниже N',
# #                     np.where((df_res3['Откл. От среднего']>d1),         
# #'Высокое откл. от кривой >'+"{:.2f}%".format(round(d1*100, 2)),
# #                     np.where((df_res3['Откл. От среднего']<d3),         
# #'Высокое откл. от кривой <'+"{:.2f}%".format(round(d3*100, 2)),
#                      np.where(((df_res3['fare-1']=='min')&(df_res3['Откл. От среднего']>0.2)&(df_res3['CLASS_NAME_x'].str.len()>1)),
# 'Открыт min RBD, откл от кривой >20%',
#                      np.where(((df_res3['fare+1']=='max')&(df_res3['numRBD_x']!=13)&((df_res3['CLASS_NAME_x'].str.len()>1))),
# 'Открыт max RBD',
#                      np.where(((df_res3['Откл. От среднего']>=-0.15)&((np.where(((df_res3['fare+1']=='max')|(df_res3['fare+1'].isna())|(df_res3['fare+1']<0)),df_res3['UT_price'],df_res3['fare+1'])+df_res3['UT_price'])<df_res3['min'])),
# 'Следующий RBD ниже мин.конкурента',
# 'Норма'))))))))))))))


                            

In [87]:
%%time
df_res3['LF']=df_res3['LF'].apply(lambda x: round(x*100, 0))
df_res3['mean LF']=df_res3['mean LF'].apply(lambda x: round(x*100, 0))
df_res3['std_dev']=df_res3['std_dev'].apply(lambda x: round(x*100, 0))
df_res3['Откл. От среднего']=df_res3['Откл. От среднего'].apply(lambda x: round(x*100, 0))


KeyError: 'Откл. От среднего'

In [84]:
%%time
df_res3=df_res3.rename(columns={'od_x':'od','booked':'bkd','available':'av','Дней до вылета':'Days',
                                'Откл. От среднего':'dLF','Изменение тарифа':'dFare','SCENARIO_NAME':'scenario','std_dev':'stdev'})

Wall time: 13 ms


In [ ]:
%%time
df_res4=df_res3[['Рейс', 'Дата вылета', 'Вылет', 'od', 'Тип ВС', 'bkd','av','% rcnt',#'rcnt_avg_fare',
                 'rbd','Days','BLF','FLF',
        'LF', 'mean LF','depLF',
       'stdev','dLF', 'UT_price', 'min', 'mean',
       'max','min_price', 'controller',
       'Категория',#'Категория2',
                 'категория откл',
       'fare-3', 'fare-2','fare-1', 'fare+1', 'fare+2','fare+3','scenario','Комментарий', 'Мин/Макс тариф?',
               'RELEVANT_COMPETITOR', 'MINUTES_BEFORE', 'MINUTES_AFTER', 'LEG_CAP',
                'dFare','snapshot','updated2','Москва/Регионы','rbd_av', 'rbd_bkd','numRBD_x', 'CLASS_NAME_x','tax','LF<mean-std',
       'LF>mean+std', 'UT<min', 'UT<mean', 'UT<max',
                 'F','Z', 'P', 'K', 'L', 'H', 'V', 'R', 'O', 'U', 'W', 'X', 'B', 'N', 'G', 'Q', 'E', 'T', 'S', 'Y',
                 'A','D','I','J','C','avg_fare','rev_tct']]

In [ ]:
%%time
df_res4=df_res4.sort_values(['Москва/Регионы','od','Рейс','Days'])
df_res4['scenario']=df_res4['scenario'].fillna('Нет сценария')
#df_res4['Дата вылета']=df_res4['Дата вылета'].replace('-', '.', inplace=True)
df_res5=df_res4.groupby(['controller','od','scenario','Категория']).agg({'Категория':'count'})
df_res5=df_res5.rename(columns={'Категория':'count'})
df_res5=df_res5.reset_index()

In [ ]:
%%time
#group flights by scenario
result=pd.pivot_table(df_res5,values='count',index=['controller','od','scenario'],columns='Категория',fill_value=0).reset_index()
result['Total']=result.iloc[:,4:15].sum(axis=1)
result['% рейсов в норме']=round(np.divide((result['Норма']),result['Total'])*100,0)
result=result.sort_values(by=['controller','od','Total'])

In [ ]:
%%time
#booking curves
df=df[~df['FLIGHT_NR'].isin([401, 402, 403, 404, 405, 406, 407, 408, 409, 412, 413, 414, 415, 416, 417, 418, 419, 420,
 421, 422, 423, 424, 425, 429, 430, 431, 432, 677,678,679,680,1008,1009,1404,1405,1426,1427,1410,1411,1402,1403,426,427,1426,1427])]
df=df[df['FLIGHT_NR']<4000]

res_curves=df.groupby(['FLIGHT_NR','od','SNAP_NR','EQUIP_ID']).agg({'BLF':'mean','FLF':'median'}).reset_index()

res_curves=res_curves.sort_values(['SNAP_NR'],ascending=False)
res_curves=res_curves[res_curves['SNAP_NR']!=-9]

res_curves=pd.pivot_table(res_curves,values=['BLF','FLF'],index=['EQUIP_ID','od'],columns='SNAP_NR').sort_values(['EQUIP_ID','od'],ascending=False).reset_index()

res_curves=res_curves.fillna(-1)
res_curves.BLF=res_curves.BLF.astype(int)
res_curves.FLF=res_curves.FLF.astype(int)

In [ ]:
%%time
#group flights by controller
res_by_controller=result.groupby('controller').agg({'Норма':'sum','Total':'sum'}).reset_index()
res_by_controller['% рейсов в норме']=round(np.divide((res_by_controller['Норма']),res_by_controller['Total'])*100,0)
res_by_controller=res_by_controller.sort_values('% рейсов в норме',ascending=False).reset_index(drop=True)

In [ ]:
df_res3[['LF<mean-std','LF>mean+std','mean LF', 'RELEVANT_COMPETITOR','min','mean','max']].describe()

In [ ]:
%%time
description={'Рейс':'Номер рейса',
 'Дата вылета':'Дата вылета',
 'Вылет':'Пункт вылета',
 'od':'Маршрут',
 'Тип ВС':'Тип воздушного судна из Rembrandt',
 'bkd':'Количество забронированных мест из Леонардо(дата обновления  - столбец updated2)',
 'av':'Количество свободных мест из Леонардо(дата обновления  - столбец updated2)',
 '% rcnt':'недавно забронированные места <7 дней до вылета за последние 24 часа, >=7 дней до вылета за последние 72 часа',
 'rbd':'Текущий класс бронирования по которому открыта продажа(дата обновления  - столбец updated2)',
 'Days':'Количество дней до вылета',
 'BLF':'% загрузки из Рембрандт(по нему закрывается сценарий -B)',
 'FLF':'прогнозный % загрузки(по нему закрывается сценарий - C)',
 'LF':'расчётный % загрузки(бронирования из Леонардо, емкость статическая)',
 'mean LF':'кривая загрузки за рейсы последних 30 дней',
 'depLF':'загрузка по вылету за последние 30 дней',
 'stdev':'стандартное отклонение от кривой загрузки',
 'dLF':'разница между LF - mean LF',
 'UT_price':'тариф Utair посчитан через цену RBD из Сирены+Сборы',
 'min':'минимальный тариф из Infare среди конкурентов R3,DP,5N,A4,ZF,WZ,U6,N4,Y7,PC Якутия,Победа,Нордавиа,Азимут,Азур,РедВингс,Урал,Нордвинд,Нордстар,Пегас',
 'mean':'минимальный тариф из Infare среди конкурентов S7,6R,YC,IO,I8,7R Сибирь,Алроса,Ямал,Ираэро,Ижавиа,Руслайн',
 'max':'минимальный тариф Аэрофлота',
 'controller':'Ответственный за рейс',
 'Категория':'Категория рейса',
 'категория откл':'10% - загрузка выше кривой от 10% до 30%, 30% - загрузка выше кривой более чем на 30%, -10% и -30% по аналогии',
 'fare-3':'-3 rbd ниже меньше текущего rbd на',
 'fare-2':'-2 rbd ниже меньше текущего rbd на',
 'fare-1':'-1 rbd ниже меньше текущего rbd на',
 'fare+1':'+1 rbd выше больше текущего rbd на',
 'fare+2':'+2 rbd выше больше текущего rbd на',
 'fare+3':'+3 rbd выше больше текущего rbd на',
 'scenario':'Имя сценария в Rembrandt для данного рейса',
 'Комментарий':'пустое поле для комментариев',
 'RELEVANT_COMPETITOR':'Сопоставимый конкурент по Rembrandt для данного рейса',
 'MINUTES_BEFORE':'Правило сравнения с конкурентами по времени, минуты до вылета',
 'MINUTES_AFTER':'Правило сравнения с конкурентами по времени, минуты после вылета',
 'LEG_CAP':'Емкость Рембрандт',
 'dFare':'Отклонение от тарифа конкурентов(если отстаем от кривой, то берется min группа тарифов, если опережаем то mean группа тарифов)',
 'snapshot':'Дата получения тарифа конкурента из Infare',
 'updated2':'Дата получения rbd, забронированных мест из Leonardo',
 'Москва/Регионы':'Москва - московские рейсы, Регионы - региональные',
 'rbd_av':'Количество свободных мест в текущем RBD',
 'rbd_bkd':'Количество забронированных мест в текущем RBD',
 'numRBD_x':'порядковый номер RBD для сортировки(F-26,Z-25,P-24 и т.д., С-1)',
 'CLASS_NAME_x':'список всех RBD из Сирены по данному рейсу',
 'tax':'признак есть ли сборы в столбце UT_price, fare - только тариф, fare+tax - тариф +сборы',
 'LF<mean-std':'если загрузка отстает от кривой за вычетом стандартного отклонения - 1, иначе 0',
 'LF>mean+std':'если загрузка опережает кривую+стандартное отклонение - 1, иначе 0',
 'UT<min':'тариф Utair меньше среди min,mean,max групп конкурентов -1, иначе 0',
 'UT<mean':'тариф Utair меньше среднего среди min,mean,max групп-конкурентов -1,иначе 0',
 'UT<max':'тариф Utair меньше максимального среди min,mean,max групп-конкурентов -1,иначе 0',
 'F':'количество забронированных мест в F',
 'Z':'количество забронированных мест в Z',
 'P':'',
 'K':'',
 'L':'',
 'H':'',
 'V':'',
 'R':'',
 'O':'',
 'U':'',
 'W':'',
 'X':'',
 'B':'',
 'N':'',
 'G':'',
 'Q':'',
 'E':'',
 'T':'',
 'S':'',
 'Y':'',
 'A':'',
 'D':'',
 'I':'',
 'J':'',
 'C':''}

In [ ]:
%%time
x=df_res4.columns.tolist()
desc= pd.DataFrame(x,columns =['Столбец']) 
desc['Описание']=desc['Столбец'].map(description)

In [ ]:
goal_c.iloc[:,5:] = goal_c.iloc[:,5:].astype(int)

In [ ]:
%%time
with pd.ExcelWriter(r'AllFutureFlights.xlsx',engine='xlsxwriter') as writer:  
    df_res4.to_excel(writer, sheet_name='Flights',header=True, index=False,encoding='utf-8-sig')
    worksheet1=writer.sheets['Flights']
    worksheet1.freeze_panes(1,3)
    result.to_excel(writer, sheet_name='Scenarios',header=True, index=False,encoding='utf-8-sig')
    res_by_controller.to_excel(writer,sheet_name='Controllers',header=True, index=False,encoding='utf-8-sig')
    goal_c.to_excel(writer,sheet_name='Curves',header=True, index=True,encoding='utf-8-sig')
    desc.to_excel(writer,sheet_name='Description',header=True, index=True,encoding='utf-8-sig')

In [ ]:
# x=df_res3[(df_res3['rbd_av']>10) & ((df_res3['категория откл']=='-30%') | (df_res3['категория откл']=='-10%')) & (df_res3['Days']>7) & (df_res3['Days']<30)].sort_values(['UT_price','od'])
# x=x.drop_duplicates(['od'], keep='first')
# x2=x[['Рейс','Вылет','Прилет', 'od','Дата вылета','UT_price','Москва/Регионы']]

In [ ]:
# x2['Вылет']=x2['Вылет'].str.replace('VKO','MOW')
# x2['Прилет']=x2['Прилет'].str.replace('VKO','MOW')

# x2=x2.merge(df_city[['code','name']],left_on='Вылет',right_on='code',how='left')
# x2=x2.rename(columns={'name':'Город вылета','code':'origin'})

# x2=x2.merge(df_city[['code','name']],left_on='Прилет',right_on='code',how='left')
# x2=x2.rename(columns={'name':'Город прилета','code':'destination'})

# exclude=['DYR-VKO','USK-VKO','NNM-VKO','KGP-VKO']
# x2=x2[~x2['od'].isin(exclude)]
# x2['deeplink']="https://www.utair.ru/booking/search/!/"+x2['Вылет']+"/"+x2['Прилет']+"/"+x2['Дата вылета'].str.slice(start=0,stop=2)+"."+x2['Дата вылета'].str.slice(start=3,stop=5)+".20"+x2['Дата вылета'].str.slice(start=6,stop=8)+"/"
# x2=x2[['Дата вылета','Город вылета','Город прилета','UT_price','Москва/Регионы','deeplink']]
# x2=x2.sort_values(by=['Москва/Регионы','UT_price'])
# x2=x2.reset_index(drop=True)
# x2=x2.head(30)


In [ ]:
# if (len(x2.index)==30) & (x2.isna().sum().sum()==0):
#     x2.to_csv(r'MinPricesPerRoute.csv',header=True, index=False,encoding='utf-8-sig')
# else:
#     print(len(x2.index))
#     print(x2.isna().sum().sum())

In [ ]:
df_res4.head()

# Делаем задачу по сравнению с Победой

In [ ]:
datetime.datetime.now().isocalendar()

In [ ]:
datetime.datetime(2021, 6, 10, 12, 39, 39).isocalendar()

In [102]:
#parametres
if datetime.datetime.now().isocalendar()[2] > 2:
    one_week = datetime.datetime.now().isocalendar()[1] - 2
    two_week = datetime.datetime.now().isocalendar()[1] - 1
    perspective_one_week = datetime.datetime.now().isocalendar()[1]
    perspective_two_week = datetime.datetime.now().isocalendar()[1] + 1
else:
    one_week = datetime.datetime.now().isocalendar()[1] - 3
    two_week = datetime.datetime.now().isocalendar()[1] - 2
    perspective_one_week = datetime.datetime.now().isocalendar()[1] - 1
    perspective_two_week = datetime.datetime.now().isocalendar()[1]
    
print(one_week,' one_week')
print(two_week,' two_week')
print(perspective_one_week,' perspective_one_week')
print(perspective_two_week,' perspective_two_week')

23  one_week
24  two_week
25  perspective_one_week
26  perspective_two_week


In [103]:
%%time
df_vnukovo_report = pd.read_excel('//utair.dom/ut/UTPA_OUD/Запорощенко/VnukovoReportData.xlsx')

Wall time: 4.71 s


In [104]:
df_vnukovo_report[:3]

,Город,Название а/к,Пассажиры,Занят. кресел,мвл/ввл,год,неделя
0,САНКТ-ПЕТЕРБУРГ,Россия,12899,0.76,ВВЛ,2018,12
1,САНКТ-ПЕТЕРБУРГ,ЮТЭЙР,5011,0.84,ВВЛ,2018,12
2,САНКТ-ПЕТЕРБУРГ,Победа,4787,0.90,ВВЛ,2018,12


In [105]:
df_vnukovo_report.shape

(27067, 7)

In [106]:
airports_utair = df_vnukovo_report[(df_vnukovo_report['Название а/к'] == 'ЮТЭЙР') &
                  #(df_vnukovo_report['год'] == 2021) &
                  ((df_vnukovo_report['неделя '] == one_week) | (df_vnukovo_report['неделя '] == two_week))
                 ]['Город'].value_counts().index

In [107]:
airports_pobeda = df_vnukovo_report[(df_vnukovo_report['Название а/к'] == 'ПОБЕДА') &
                  #(df_vnukovo_report['год'] == 2021) &
                  ((df_vnukovo_report['неделя '] == one_week) | (df_vnukovo_report['неделя '] == two_week))
                 ]['Город'].value_counts().index

In [108]:
cities = set(airports_utair)&set(airports_pobeda)

In [109]:
df_vnukovo_report['Город'][0] in cities

True

In [110]:
df_vnukovo_report['всего_кресел'] = df_vnukovo_report['Пассажиры']/df_vnukovo_report['Занят. кресел']

In [111]:
df_vnukovo_svod_utair = df_vnukovo_report[
    ((df_vnukovo_report['неделя ']==one_week) | (df_vnukovo_report['неделя ']==two_week)) & 
    (df_vnukovo_report['год'] == 2021) &
    (df_vnukovo_report['Название а/к'] == 'ЮТЭЙР') &
    (df_vnukovo_report['Город'].isin(cities))].pivot_table(
    values = ['Пассажиры','всего_кресел'], columns = 'неделя ', index = 'Город', aggfunc = 'sum')

df_vnukovo_svod_utair = df_vnukovo_svod_utair['Пассажиры']/df_vnukovo_svod_utair['всего_кресел']

In [112]:
df_vnukovo_svod_pobeda = df_vnukovo_report[
    ((df_vnukovo_report['неделя ']==one_week) | (df_vnukovo_report['неделя ']==two_week)) & 
    (df_vnukovo_report['год'] == 2021) &
    (df_vnukovo_report['Название а/к'] == 'ПОБЕДА') &
    (df_vnukovo_report['Город'].isin(cities))].pivot_table(
    values = ['Пассажиры','всего_кресел'], columns = 'неделя ', index = 'Город', aggfunc = 'sum')

df_vnukovo_svod_pobeda = df_vnukovo_svod_pobeda['Пассажиры']/df_vnukovo_svod_pobeda['всего_кресел']

In [113]:
df_vnukovo_svod_pobeda = df_vnukovo_svod_pobeda.reset_index().rename(
    columns = {one_week:'{} ПОБЕДА'.format(one_week),two_week:'{} ПОБЕДА'.format(two_week)})
df_vnukovo_svod_utair = df_vnukovo_svod_utair.reset_index().rename(
    columns = {one_week:'{} ЮТЭЙР'.format(one_week),two_week:'{} ЮТЭЙР'.format(two_week)})

In [114]:
# df_vnukovo_svod_utair[#(df_vnukovo_report['год']==2021) &
#                   (df_vnukovo_svod_utair['Город']=='БЕРЛИН')&
#                   (df_vnukovo_svod_utair['неделя ']==22)
    
    
# ]

In [115]:
print(df_vnukovo_svod_pobeda.shape)
df_vnukovo_svod_pobeda[:2]

(21, 3)


неделя,Город,23 ПОБЕДА,24 ПОБЕДА
0,АНАПА,0.925,0.948
1,БЕРЛИН,0.913,0.891


In [116]:
print(df_vnukovo_svod_utair.shape)
df_vnukovo_svod_utair[:2]

(21, 3)


неделя,Город,23 ЮТЭЙР,24 ЮТЭЙР
0,АНАПА,0.672,0.865
1,ВЛАДИКАВКАЗ,0.915,0.875


# Сделали LF DP и LF UT на 21 и 22ую неделю

In [117]:
main = df_vnukovo_svod_utair.merge(df_vnukovo_svod_pobeda, how = 'left', on = 'Город')
main[[main.columns[0],main.columns[1],main.columns[3],main.columns[2],main.columns[4]]]

неделя,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА
0,АНАПА,0.672,0.925,0.865,0.948
1,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950
2,ГЕЛЕНДЖИК,0.719,0.930,0.860,0.953
3,КАЗАНЬ,0.937,0.969,0.915,0.966
4,КАЛИНИНГРАД,0.924,0.976,0.899,0.977
5,КРАСНОДАР,0.883,0.947,0.921,0.948
6,КРАСНОЯРСК,0.902,0.974,0.955,0.967
7,КУРГАН,0.885,0.935,0.924,0.914
8,МАХАЧКАЛА,0.926,0.967,0.930,0.961
9,МИНЕРАЛЬНЫЕ ВОДЫ,0.847,0.961,0.918,0.934


In [118]:
main.columns

Index(['Город', '23 ЮТЭЙР', '24 ЮТЭЙР', '23 ПОБЕДА', '24 ПОБЕДА'], dtype='object', name='неделя ')

In [119]:
%%time
df_kpi = pd.read_excel(
    '//10.96.37.74/report$/1_Main factors/Ежедневный отчет (Основные показатели)/01_Основные показатели (ежедневный отчет).xlsx',
    sheet_name="Данные"
    )
#//10.96.37.74/report$/1_Main factors/Ежедневный отчет (Основные показатели)

Wall time: 6min


In [120]:
#df_kpi1 = df_kpi.copy()

In [121]:
#df_kpi = df_kpi1

In [122]:
ct = 'ИРКУТСК'[:1] + 'ИРКУТСК'[1:].lower()

In [123]:
df_kpi.loc[df_kpi['Аэропорт вылета (город)'] == ct]['Аэропорт вылета (код)'][0:1].values[0]

'IKT'

In [124]:
def init_city(x):
    if x == 'БАРИ':
        return 'BRI'
    elif x == 'МИНЕРАЛЬНЫЕ ВОДЫ':
        return 'MRV'
    elif x == 'НАЛЬЧИК':
        return 'NAL'
    elif x == 'РОСТОВ-НА-ДОНУ':
        return 'ROV'
    elif x == 'САНКТ-ПЕТЕРБУРГ':
        return 'LED'
    else:
        #print(x)
        ct = x[:1] + x[1:].lower()
        #print(ct)
        a = df_kpi.loc[df_kpi['Аэропорт вылета (город)'] == ct]['Аэропорт вылета (код)'][0:1].values[0]
        #print(a)
        #print('\n\n')
        return a

In [125]:
main['Code'] = main['Город'].apply(lambda x: init_city(x))
main['Code2'] = 'VKO'
main['Маршрут'] = main[['Code','Code2']].apply(lambda x: '-'.join(sorted(x)),axis=1)
main['Направление'] = main[['Code','Code2']].apply(lambda x: [x[0] + '-' + x[1],x[1] + '-' + x[0]], axis=1)

In [126]:
main = main.set_index(main.columns.to_list()[:-1])['Направление'].apply(
    pd.Series).stack().reset_index().drop('level_8', axis=1).rename(columns={0:'Направление'})

In [127]:
main['Вылет'] = main['Направление'].apply(lambda x: x.split('-')[0])

In [128]:
main[:5]

,Город,23 ЮТЭЙР,24 ЮТЭЙР,23 ПОБЕДА,24 ПОБЕДА,Code,Code2,Маршрут,Направление,Вылет
0,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,AAQ-VKO,AAQ
1,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,VKO-AAQ,VKO
2,ВЛАДИКАВКАЗ,0.915,0.875,0.951,0.950,OGZ,VKO,OGZ-VKO,OGZ-VKO,OGZ
3,ВЛАДИКАВКАЗ,0.915,0.875,0.951,0.950,OGZ,VKO,OGZ-VKO,VKO-OGZ,VKO
4,ГЕЛЕНДЖИК,0.719,0.860,0.930,0.953,GDZ,VKO,GDZ-VKO,GDZ-VKO,GDZ


In [129]:
df_kpi['Дата рейса Local'][0]

Timestamp('2019-12-23 00:00:00')

In [130]:
df_kpi['type'] = df_kpi['Дата рейса Local'].apply(lambda x: type(x))

In [131]:
#df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'][74817]

In [132]:
#pd.TimedeltaIndex(df_kpi['Дата рейса Local'], unit='d') + dt.datetime(1900,1,1)

In [133]:
#df_kpi['Дата рейса Local'] = pd.to_datetime(df_kpi['Дата рейса Local'],  format='%Y-%m-%d %H:%M:%S')

In [134]:
import datetime

In [135]:
#import datetime as dt
#df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'] = 
#pd.TimedeltaIndex(df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'], unit='d') + datetime.datetime(1900,1,1)
# df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'] = df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'].apply(
#     lambda x: pd.Timedelta(x, unit='d') + datetime.datetime(1900,1,1))

df_kpi.loc[df_kpi['type'] == int, 'Дата рейса Local'] = df_kpi[(df_kpi['type'] == int)]['Дата рейса Local'].apply(
    lambda x: pd.Timedelta(x, unit='d') + datetime.datetime(1900,1,1))

In [136]:
df_kpi['Дата рейса Local'] = pd.to_datetime(df_kpi['Дата рейса Local'],  format='%Y-%m-%d %H:%M:%S')

In [137]:

df_kpi_filtr = df_kpi[(df_kpi['Дата рейса Local'].dt.year == 2021) &  #.dt
                      (df_kpi['Связь'].isin(main['Маршрут'].values))]

#df_kpi_filtr['Дата рейса Local'] = pd.to_datetime(df_kpi_filtr['Дата рейса Local'],  format='%Y-%m-%d %H:%M:%S')
df_kpi_filtr['Неделя'] = df_kpi_filtr['Дата рейса Local'].apply(lambda x: x.isocalendar()[1])
df_kpi_filtr = df_kpi_filtr[(df_kpi_filtr['Неделя'].isin([one_week,two_week]))]

df_kpi_filtr = df_kpi_filtr[['Перевезено пассажиров, всего (сумма П1:П2)',
        'Реальная емкость ВС, всего (сумма Е1:Е2)',
        'Связь',
        'Аэропорт вылета (код)', 
        #'Дата рейса Local', 
        'Неделя']]

print(df_kpi_filtr.shape)
df_kpi_filtr[:5]


(1220, 5)


<ipython-input-137-901dc1f376eb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kpi_filtr['Неделя'] = df_kpi_filtr['Дата рейса Local'].apply(lambda x: x.isocalendar()[1])


,"Перевезено пассажиров, всего (сумма П1:П2)","Реальная емкость ВС, всего (сумма Е1:Е2)",Связь,Аэропорт вылета (код),Неделя
74698,123,159.0,KRO-VKO,VKO,23
74699,147,159.0,KUF-VKO,VKO,23
74700,32,159.0,AAQ-VKO,AAQ,23
74705,115,186.0,SGC-VKO,VKO,23
74707,170,186.0,KGD-VKO,KGD,23


In [138]:
#df_kpi_filtr['Дата рейса Local'].value_counts()

In [139]:
df_kpi_filtr_svod = df_kpi_filtr.pivot_table(
    values = ['Перевезено пассажиров, всего (сумма П1:П2)',
              'Реальная емкость ВС, всего (сумма Е1:Е2)'], index = ['Связь','Аэропорт вылета (код)'], 
    columns = 'Неделя', aggfunc = 'sum')
df_kpi_filtr_svod = df_kpi_filtr_svod['Перевезено пассажиров, всего (сумма П1:П2)']/df_kpi_filtr_svod['Реальная емкость ВС, всего (сумма Е1:Е2)']
df_kpi_filtr_svod.reset_index(inplace = True)

In [140]:
df_kpi_filtr_svod[:1]

Неделя,Связь,Аэропорт вылета (код),23,24
0,AAQ-VKO,AAQ,0.397411,0.749548


In [141]:
main[:1]

,Город,23 ЮТЭЙР,24 ЮТЭЙР,23 ПОБЕДА,24 ПОБЕДА,Code,Code2,Маршрут,Направление,Вылет
0,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,AAQ-VKO,AAQ


In [142]:
main = main.merge(df_kpi_filtr_svod, how = 'left', 
           left_on= ['Маршрут','Вылет'], 
           right_on= ['Связь','Аэропорт вылета (код)'])
main#[:2]

,Город,23 ЮТЭЙР,24 ЮТЭЙР,23 ПОБЕДА,24 ПОБЕДА,Code,Code2,Маршрут,Направление,Вылет,Связь,Аэропорт вылета (код),23,24
0,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,AAQ-VKO,AAQ,AAQ-VKO,AAQ,0.397411,0.749548
1,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,VKO-AAQ,VKO,AAQ-VKO,VKO,0.943905,0.885038
2,ВЛАДИКАВКАЗ,0.915,0.875,0.951,0.950,OGZ,VKO,OGZ-VKO,OGZ-VKO,OGZ,OGZ-VKO,OGZ,0.885314,0.864711
3,ВЛАДИКАВКАЗ,0.915,0.875,0.951,0.950,OGZ,VKO,OGZ-VKO,VKO-OGZ,VKO,OGZ-VKO,VKO,0.944719,0.885086
4,ГЕЛЕНДЖИК,0.719,0.860,0.930,0.953,GDZ,VKO,GDZ-VKO,GDZ-VKO,GDZ,GDZ-VKO,GDZ,0.505294,0.779473
5,ГЕЛЕНДЖИК,0.719,0.860,0.930,0.953,GDZ,VKO,GDZ-VKO,VKO-GDZ,VKO,GDZ-VKO,VKO,0.928844,0.945446
6,КАЗАНЬ,0.937,0.915,0.969,0.966,KZN,VKO,KZN-VKO,KZN-VKO,KZN,KZN-VKO,KZN,0.929043,0.922500
7,КАЗАНЬ,0.937,0.915,0.969,0.966,KZN,VKO,KZN-VKO,VKO-KZN,VKO,KZN-VKO,VKO,0.944719,0.916667
8,КАЛИНИНГРАД,0.924,0.899,0.976,0.977,KGD,VKO,KGD-VKO,KGD-VKO,KGD,KGD-VKO,KGD,0.903530,0.838513
9,КАЛИНИНГРАД,0.924,0.899,0.976,0.977,KGD,VKO,KGD-VKO,VKO-KGD,VKO,KGD-VKO,VKO,0.943811,0.871448


In [143]:
#df_res4

In [144]:
[perspective_one_week,perspective_two_week]

[25, 26]

In [145]:
svod_from_res = df_res4[(df_res4['od'].isin(main['Маршрут'].values))][['Дата вылета','Вылет','od','bkd','av']]
svod_from_res['Неделя'] = svod_from_res['Дата вылета'].apply(lambda x: x.isocalendar()[1])
svod_from_res['Объем'] = svod_from_res['bkd'] + svod_from_res['av']
svod_from_res = svod_from_res[(svod_from_res['Неделя'].isin([perspective_one_week,perspective_two_week]))]
svod_from_res[:3]
# 'bkd':'Количество забронированных мест из Леонардо(дата обновления  - столбец updated2)'
# 'av':'Количество свободных мест из Леонардо(дата обновления  - столбец updated2)'

,Дата вылета,Вылет,od,bkd,av,Неделя,Объем
7976,2021-06-24,VKO,AAQ-VKO,117.0,72.0,25,189.0
5336,2021-06-25,VKO,AAQ-VKO,137.0,52.0,25,189.0
2737,2021-06-26,VKO,AAQ-VKO,118.0,71.0,25,189.0


In [146]:
main['Маршрут'].values

array(['AAQ-VKO', 'AAQ-VKO', 'OGZ-VKO', 'OGZ-VKO', 'GDZ-VKO', 'GDZ-VKO',
       'KZN-VKO', 'KZN-VKO', 'KGD-VKO', 'KGD-VKO', 'KRR-VKO', 'KRR-VKO',
       'KJA-VKO', 'KJA-VKO', 'KRO-VKO', 'KRO-VKO', 'MCX-VKO', 'MCX-VKO',
       'MRV-VKO', 'MRV-VKO', 'MMK-VKO', 'MMK-VKO', 'IGT-VKO', 'IGT-VKO',
       'ROV-VKO', 'ROV-VKO', 'KUF-VKO', 'KUF-VKO', 'LED-VKO', 'LED-VKO',
       'AER-VKO', 'AER-VKO', 'STW-VKO', 'STW-VKO', 'SGC-VKO', 'SGC-VKO',
       'TJM-VKO', 'TJM-VKO', 'ULV-VKO', 'ULV-VKO', 'UFA-VKO', 'UFA-VKO'],
      dtype=object)

In [147]:
svod_from_res.isna().sum()

Дата вылета    0
Вылет          0
od             0
bkd            6
av             6
Неделя         0
Объем          6
dtype: int64

In [148]:
svod_from_res = svod_from_res.pivot_table(
    values = ['bkd','av','Объем'], index = ['Вылет','od'], columns = 'Неделя', aggfunc = 'sum')#.reset_index()

# Сделали перспективную загрузку

In [149]:
svod_from_res[:5]

av             bkd           Объем        
Неделя            25      26      25      26      25      26
Вылет od                                                    
AAQ   AAQ-VKO  515.0  1045.0  2497.0  4055.0  3012.0  5100.0
AER   AER-VKO  222.0   892.0  2809.0  4339.0  3031.0  5231.0
GDZ   GDZ-VKO  256.0   818.0  1104.0  1612.0  1360.0  2430.0
IGT   IGT-VKO  176.0   759.0   584.0   539.0   760.0  1298.0
KGD   KGD-VKO  562.0  1465.0  1425.0  2339.0  1987.0  3804.0

In [150]:
svod_from_res_good = svod_from_res['bkd']/svod_from_res['Объем']

In [151]:
svod_from_res_good = svod_from_res_good.merge(svod_from_res['av'], how = 'left', on = ['Вылет','od']).reset_index()
svod_from_res_good[:3]

Неделя,Вылет,od,25_x,26_x,25_y,26_y
0,AAQ,AAQ-VKO,0.829017,0.795098,515.0,1045.0
1,AER,AER-VKO,0.926757,0.829478,222.0,892.0
2,GDZ,GDZ-VKO,0.811765,0.663374,256.0,818.0


# Рассчитываем Median Fare

In [152]:
%%time
df_fares_dp = pd.read_csv('//utair.dom/ut/UTPA_OUD/Запорощенко/FaresOtherAirlines.csv')

Wall time: 3min 30s


In [153]:
df_fares_dp = df_fares_dp[(df_fares_dp['carrier']=='DP')]

In [154]:
%%time
df_fares_ut = pd.read_csv('//utair.dom/ut/UTPA_OUD/Запорощенко/FaresUtair.csv')

Wall time: 45.6 s


In [155]:
print(df_fares_dp.shape)
df_fares_dp[:2]

(583316, 14)


,index,carrier,origin,destination,flight_no,flight_datetime,price,snapshot,pos,flight_date,departure_time,o,d,od
0,2,DP,AER,UFA,343,2021-08-11 14:15:00,6147,2021-06-22,RU,2021-08-11,14:15:00,AER,UFA,AERUFA
1,3,DP,VKO,AAQ,145,2021-08-22 19:05:00,3526,2021-06-22,RU,2021-08-22,19:05:00,VKO,AAQ,AAQVKO


In [156]:
print(df_fares_ut.shape)
df_fares_ut[:2]

(1055228, 14)


,index,carrier,origin,destination,flight_no,flight_datetime,price,snapshot,pos,flight_date,departure_time,o,d,od
0,0,UT,MCX,VKO,560,2021-09-07 06:10:00,4385,2021-06-22,RU,2021-09-07,06:10:00,MCX,VKO,MCXVKO
1,1,UT,UFA,TJM,1126,2021-08-20 22:45:00,4685,2021-06-22,RU,2021-08-20,22:45:00,UFA,TJM,TJMUFA


In [157]:
df_fares_dp['flight_datetime'].min()

'2021-04-25 11:10:00'

In [158]:
df_fares_dp['flight_datetime'] = pd.to_datetime(df_fares_dp['flight_datetime'],  format='%Y-%m-%d %H:%M:%S')
df_fares_ut['flight_datetime'] = pd.to_datetime(df_fares_ut['flight_datetime'],  format='%Y-%m-%d %H:%M:%S')

In [159]:
%time
df_fares_dp['Неделя'] = df_fares_dp['flight_datetime'].apply(lambda x: x.isocalendar()[1])
df_fares_ut['Неделя'] = df_fares_ut['flight_datetime'].apply(lambda x: x.isocalendar()[1])

Wall time: 0 ns


In [160]:
list_od = [i.replace('-','') for i in main['Маршрут'].values]

In [161]:
len(list_od)

42

In [162]:
df_fares_dp_good = df_fares_dp[(df_fares_dp['Неделя'].isin([one_week,two_week])) & (df_fares_dp['od'].isin(list_od))]
df_fares_ut_good = df_fares_ut[(df_fares_ut['Неделя'].isin([one_week,two_week])) & (df_fares_ut['od'].isin(list_od))]

In [163]:
df_fares_ut_good['Неделя'].value_counts()

24    33210
23    28524
Name: Неделя, dtype: int64

In [164]:
df_fares_dp_good['Неделя'].value_counts()

24    35346
23    30309
Name: Неделя, dtype: int64

In [165]:
df_fares_dp_good['od'].value_counts().shape

(19,)

In [166]:
df_fares_dp_good[0:2]

,index,carrier,origin,destination,flight_no,flight_datetime,price,snapshot,pos,flight_date,departure_time,o,d,od,Неделя
139688,9047,DP,VKO,KRO,495,2021-06-20 23:50:00,2994,2021-06-20,RU,2021-06-20,23:50:00,VKO,KRO,KROVKO,24
177999,58761,DP,STW,VKO,186,2021-06-20 17:40:00,7986,2021-06-20,RU,2021-06-20,17:40:00,STW,VKO,STWVKO,24


# Fares для Победы и Ютейр сводная

In [167]:
df_fares_dp_good_svod = df_fares_dp_good.pivot_table(
    values = 'price', index = ['od','o'], columns = 'Неделя', aggfunc = 'median').reset_index()
print(df_fares_dp_good_svod.shape)
df_fares_dp_good_svod[0:3]

(38, 4)


Неделя,od,o,23,24
0,AAQVKO,AAQ,2495.0,3500.0
1,AAQVKO,VKO,6005.0,5114.0
2,AERVKO,AER,2508.0,3502.0


In [168]:
df_fares_ut_good_svod = df_fares_ut_good.pivot_table(
    values = 'price', index = ['od','o'], columns = 'Неделя', aggfunc = 'median').reset_index()
print(df_fares_ut_good_svod.shape)
df_fares_ut_good_svod[0:3]

(42, 4)


Неделя,od,o,23,24
0,AAQVKO,AAQ,3255,4855
1,AAQVKO,VKO,7355,7355
2,AERVKO,AER,3085,4785


# Fares для Победы и Ютейр сводная дата продажи - последний день

In [169]:
df_fares_ut_good[:1]

,index,carrier,origin,destination,flight_no,flight_datetime,price,snapshot,pos,flight_date,departure_time,o,d,od,Неделя
44474,19747,UT,AAQ,VKO,458,2021-06-20 01:30:00,2655,2021-06-20,RU,2021-06-20,01:30:00,AAQ,VKO,AAQVKO,24


In [170]:
#df_fares_ut_good['snapshot'][211504]

In [171]:
# one_day = (datetime.datetime.now() - datetime.timedelta(days = 1)).strftime('%Y-%m-%d')
# one_day

In [172]:
#df_fares_dp_good['snapshot'] = pd.to_datetime(df_fares_dp_good['snapshot'],  format='%Y-%m-%d %H:%M:%S')
#df_fares_ut_good['snapshot'] = pd.to_datetime(df_fares_ut_good['snapshot'],  format='%Y-%m-%d %H:%M:%S')

In [173]:
one_day = df_fares_ut_good['snapshot'].value_counts().index.sort_values()[-1]

In [174]:
df_fares_ut_good_one_day = df_fares_ut[(df_fares_ut['snapshot']==one_day) & 
                                       (df_fares_ut['Неделя'].isin([perspective_one_week])) & 
                                       (df_fares_ut['od'].isin(list_od))]
df_fares_dp_good_one_day = df_fares_dp[(df_fares_dp['snapshot']==one_day) & 
                                       (df_fares_dp['Неделя'].isin([perspective_one_week])) & 
                                       (df_fares_dp['od'].isin(list_od))]

In [175]:
df_fares_ut_good_one_day = df_fares_ut_good_one_day.pivot_table(
    values = 'price', index = ['od','o'], columns = 'Неделя', aggfunc = 'median').reset_index()
print(df_fares_ut_good_one_day.shape)
df_fares_ut_good_one_day[:3]

(42, 3)


Неделя,od,o,25
0,AAQVKO,AAQ,3655
1,AAQVKO,VKO,4655
2,AERVKO,AER,4085


In [176]:
df_fares_dp_good_one_day = df_fares_dp_good_one_day.pivot_table(
    values = 'price', index = ['od','o'], columns = 'Неделя', aggfunc = 'median').reset_index()
print(df_fares_dp_good_one_day.shape)
df_fares_dp_good_one_day[:3]

(38, 3)


Неделя,od,o,25
0,AAQVKO,AAQ,3743.0
1,AAQVKO,VKO,2994.0
2,AERVKO,AER,3493.0


# Соединение таблиц

In [177]:
main[:3] # загрузка в сравнении

,Город,23 ЮТЭЙР,24 ЮТЭЙР,23 ПОБЕДА,24 ПОБЕДА,Code,Code2,Маршрут,Направление,Вылет,Связь,Аэропорт вылета (код),23,24
0,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,AAQ-VKO,AAQ,AAQ-VKO,AAQ,0.397411,0.749548
1,АНАПА,0.672,0.865,0.925,0.948,AAQ,VKO,AAQ-VKO,VKO-AAQ,VKO,AAQ-VKO,VKO,0.943905,0.885038
2,ВЛАДИКАВКАЗ,0.915,0.875,0.951,0.950,OGZ,VKO,OGZ-VKO,OGZ-VKO,OGZ,OGZ-VKO,OGZ,0.885314,0.864711


In [178]:
otv = main[[main.columns[0],main.columns[1],main.columns[3],main.columns[2],main.columns[4],main.columns[12],main.columns[13],main.columns[7],main.columns[11]]]

In [179]:
otv = otv.rename(columns = {one_week:'{} ЮТЭЙР (направления)'.format(one_week),
                            two_week:'{} ЮТЭЙР (направления)'.format(two_week)})

In [180]:
otv[:3]

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код)
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQ-VKO,AAQ
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQ-VKO,VKO
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZ-VKO,OGZ


In [181]:
otv['Маршрут'] = otv['Маршрут'].apply(lambda x: x.replace('-',''))
print(otv.shape)
otv[:3]

(42, 9)


,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код)
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ


In [182]:
df_fares_dp_good_svod[:3] # fares Победа

Неделя,od,o,23,24
0,AAQVKO,AAQ,2495.0,3500.0
1,AAQVKO,VKO,6005.0,5114.0
2,AERVKO,AER,2508.0,3502.0


In [183]:
df_fares_dp_good_svod

Неделя,od,o,23,24
0,AAQVKO,AAQ,2495.0,3500.0
1,AAQVKO,VKO,6005.0,5114.0
2,AERVKO,AER,2508.0,3502.0
3,AERVKO,VKO,5402.0,5393.0
4,GDZVKO,GDZ,2489.0,3500.0
5,GDZVKO,VKO,6003.0,5980.0
6,IGTVKO,IGT,2750.0,2750.0
7,IGTVKO,VKO,3639.0,3150.0
8,KGDVKO,KGD,2752.0,3637.0
9,KGDVKO,VKO,3649.0,3651.0


In [184]:
otv1 = otv.merge(df_fares_dp_good_svod, how = 'left', 
                 left_on = ['Маршрут','Аэропорт вылета (код)'], 
                 right_on= ['od','o'])

In [185]:
otv1.shape

(42, 13)

In [186]:
otv1[:3]

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od,o,23,24
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0


In [187]:
otv2 = otv1.merge(df_fares_ut_good_svod, how = 'left', 
                  left_on = ['Маршрут','Аэропорт вылета (код)'], 
                  right_on= ['od','o'])

In [188]:
otv2.shape

(42, 17)

In [189]:
otv2[:3]

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od_x,o_x,23_x,24_x,od_y,o_y,23_y,24_y
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0,AAQVKO,AAQ,3255,4855
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0,AAQVKO,VKO,7355,7355
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0,OGZVKO,OGZ,2585,2985


In [190]:
otv2['Вылет'] = otv2['Аэропорт вылета (код)']#.apply(lambda x: x.split('-')[0])
otv2[:3]

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od_x,o_x,23_x,24_x,od_y,o_y,23_y,24_y,Вылет
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0,AAQVKO,AAQ,3255,4855,AAQ
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0,AAQVKO,VKO,7355,7355,VKO
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0,OGZVKO,OGZ,2585,2985,OGZ


In [191]:
svod_from_res_good[:3]

Неделя,Вылет,od,25_x,26_x,25_y,26_y
0,AAQ,AAQ-VKO,0.829017,0.795098,515.0,1045.0
1,AER,AER-VKO,0.926757,0.829478,222.0,892.0
2,GDZ,GDZ-VKO,0.811765,0.663374,256.0,818.0


In [192]:
#svod_from_res1 = svod_from_res.T.reset_index().T# перспективная загрузка
svod_from_res_good1 = svod_from_res_good.rename(columns = {'od':'Маршрут',
                                                           '{}_x'.format(perspective_one_week):'{} Процент загрузки'.format(perspective_one_week),
                                                           '{}_x'.format(perspective_two_week):'{} Процент загрузки'.format(perspective_two_week),
                                                           '{}_y'.format(perspective_one_week):'{} Количество свободных мест'.format(perspective_one_week),
                                                           '{}_y'.format(perspective_two_week):'{} Количество свободных мест'.format(perspective_two_week)})
svod_from_res_good1 = svod_from_res_good1[2:]
svod_from_res_good1['Маршрут'] = svod_from_res_good1['Маршрут'].apply(lambda x: x.replace('-',''))
svod_from_res_good1[:3]
# 'bkd':'Количество забронированных мест из Леонардо(дата обновления  - столбец updated2)'
# 'av':'Количество свободных мест из Леонардо(дата обновления  - столбец updated2)'


Неделя,Вылет,Маршрут,25 Процент загрузки,26 Процент загрузки,25 Количество свободных мест,26 Количество свободных мест
2,GDZ,GDZVKO,0.811765,0.663374,256.0,818.0
3,IGT,IGTVKO,0.768421,0.415254,176.0,759.0
4,KGD,KGDVKO,0.717162,0.614879,562.0,1465.0


In [193]:
otv3 = otv2.merge(svod_from_res_good1, how = 'left', on = ['Маршрут','Вылет'])

In [194]:
print(otv3.shape)
otv3[:5]

(42, 22)


,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od_x,o_x,23_x,24_x,od_y,o_y,23_y,24_y,Вылет,25 Процент загрузки,26 Процент загрузки,25 Количество свободных мест,26 Количество свободных мест
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0,AAQVKO,AAQ,3255,4855,AAQ,NaN,NaN,NaN,NaN
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0,AAQVKO,VKO,7355,7355,VKO,0.762616,0.787108,715.0,1080.0
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0,OGZVKO,OGZ,2585,2985,OGZ,0.824728,0.432515,129.0,740.0
3,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.944719,0.885086,OGZVKO,VKO,OGZVKO,VKO,3706.0,4184.0,OGZVKO,VKO,3585,2985,VKO,0.919837,0.645929,59.0,461.0
4,ГЕЛЕНДЖИК,0.719,0.930,0.860,0.953,0.505294,0.779473,GDZVKO,GDZ,GDZVKO,GDZ,2489.0,3500.0,GDZVKO,GDZ,3185,5385,GDZ,0.811765,0.663374,256.0,818.0


In [195]:
otv4 = otv3.merge(df_fares_ut_good_one_day, how = 'left', left_on= ['Маршрут','Вылет'], right_on= ['od','o'])
# fares дата продажи последний день из снэпшота Ютэйр

In [196]:
otv4[:5]

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od_x,o_x,23_x,24_x,od_y,o_y,23_y,24_y,Вылет,25 Процент загрузки,26 Процент загрузки,25 Количество свободных мест,26 Количество свободных мест,od,o,25
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0,AAQVKO,AAQ,3255,4855,AAQ,NaN,NaN,NaN,NaN,AAQVKO,AAQ,3655
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0,AAQVKO,VKO,7355,7355,VKO,0.762616,0.787108,715.0,1080.0,AAQVKO,VKO,4655
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0,OGZVKO,OGZ,2585,2985,OGZ,0.824728,0.432515,129.0,740.0,OGZVKO,OGZ,2985
3,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.944719,0.885086,OGZVKO,VKO,OGZVKO,VKO,3706.0,4184.0,OGZVKO,VKO,3585,2985,VKO,0.919837,0.645929,59.0,461.0,OGZVKO,VKO,6585
4,ГЕЛЕНДЖИК,0.719,0.930,0.860,0.953,0.505294,0.779473,GDZVKO,GDZ,GDZVKO,GDZ,2489.0,3500.0,GDZVKO,GDZ,3185,5385,GDZ,0.811765,0.663374,256.0,818.0,GDZVKO,GDZ,3185


In [197]:
otv5 = otv4.merge(df_fares_dp_good_one_day, how = 'left', left_on= ['Маршрут','Вылет'], right_on= ['od','o'])
# fares дата продажи последний день из снэпшота Победа

In [198]:
otv5

,Город,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),Маршрут,Аэропорт вылета (код),od_x,o_x,23_x,24_x,od_y,o_y,23_y,24_y,Вылет,25 Процент загрузки,26 Процент загрузки,25 Количество свободных мест,26 Количество свободных мест,od_x,o_x,25_x,od_y,o_y,25_y
0,АНАПА,0.672,0.925,0.865,0.948,0.397411,0.749548,AAQVKO,AAQ,AAQVKO,AAQ,2495.0,3500.0,AAQVKO,AAQ,3255,4855,AAQ,NaN,NaN,NaN,NaN,AAQVKO,AAQ,3655,AAQVKO,AAQ,3743.0
1,АНАПА,0.672,0.925,0.865,0.948,0.943905,0.885038,AAQVKO,VKO,AAQVKO,VKO,6005.0,5114.0,AAQVKO,VKO,7355,7355,VKO,0.762616,0.787108,715.0,1080.0,AAQVKO,VKO,4655,AAQVKO,VKO,2994.0
2,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.885314,0.864711,OGZVKO,OGZ,OGZVKO,OGZ,2300.5,2705.0,OGZVKO,OGZ,2585,2985,OGZ,0.824728,0.432515,129.0,740.0,OGZVKO,OGZ,2985,OGZVKO,OGZ,2695.0
3,ВЛАДИКАВКАЗ,0.915,0.951,0.875,0.950,0.944719,0.885086,OGZVKO,VKO,OGZVKO,VKO,3706.0,4184.0,OGZVKO,VKO,3585,2985,VKO,0.919837,0.645929,59.0,461.0,OGZVKO,VKO,6585,OGZVKO,VKO,4192.0
4,ГЕЛЕНДЖИК,0.719,0.930,0.860,0.953,0.505294,0.779473,GDZVKO,GDZ,GDZVKO,GDZ,2489.0,3500.0,GDZVKO,GDZ,3185,5385,GDZ,0.811765,0.663374,256.0,818.0,GDZVKO,GDZ,3185,GDZVKO,GDZ,3493.0
5,ГЕЛЕНДЖИК,0.719,0.930,0.860,0.953,0.928844,0.945446,GDZVKO,VKO,GDZVKO,VKO,6003.0,5980.0,GDZVKO,VKO,7185,6385,VKO,0.783571,0.706996,303.0,712.0,GDZVKO,VKO,4185,GDZVKO,VKO,4492.0
6,КАЗАНЬ,0.937,0.969,0.915,0.966,0.929043,0.922500,KZNVKO,KZN,KZNVKO,KZN,2599.0,3138.0,KZNVKO,KZN,2285,2285,KZN,0.857895,0.608271,108.0,521.0,KZNVKO,KZN,3885,KZNVKO,KZN,2869.5
7,КАЗАНЬ,0.937,0.969,0.915,0.966,0.944719,0.916667,KZNVKO,VKO,KZNVKO,VKO,2599.0,2602.0,KZNVKO,VKO,2285,2285,VKO,0.826316,0.632331,132.0,489.0,KZNVKO,VKO,3285,KZNVKO,VKO,3144.0
8,КАЛИНИНГРАД,0.924,0.976,0.899,0.977,0.903530,0.838513,KGDVKO,KGD,KGDVKO,KGD,2752.0,3637.0,KGDVKO,KGD,3585,4285,KGD,0.717162,0.614879,562.0,1465.0,KGDVKO,KGD,3585,KGDVKO,KGD,3144.0
9,КАЛИНИНГРАД,0.924,0.976,0.899,0.977,0.943811,0.871448,KGDVKO,VKO,KGDVKO,VKO,3649.0,3651.0,KGDVKO,VKO,4285,4285,VKO,0.836172,0.717140,327.0,1076.0,KGDVKO,VKO,3585,KGDVKO,VKO,3144.0


In [199]:
try:
    otv6 = otv5[['Аэропорт вылета (код)',
                 'Маршрут', 
                 '{} ЮТЭЙР'.format(one_week),
                 '{} ПОБЕДА'.format(one_week),
                 '{} ЮТЭЙР'.format(two_week),
                 '{} ПОБЕДА'.format(two_week), 
                 '{} ЮТЭЙР (направления)'.format(one_week),
                 '{} ЮТЭЙР (направления)'.format(two_week),
                 '{}_x'.format(one_week), # Median Fare Победа
                 '{}_y'.format(one_week), # Median Fare Ютэйр
                 '{}_x'.format(two_week), # Median Fare Победа
                 '{}_y'.format(two_week), # Median Fare Ютэйр            
                 '{} Процент загрузки'.format(perspective_one_week), # Процент загрузки                      
                 '{} Процент загрузки'.format(perspective_two_week), # Процент загрузки 
                 '{} Количество свободных мест'.format(perspective_one_week), # Количество свободных мест                      
                 '{} Количество свободных мест'.format(perspective_two_week), # Количество свободных мест                
                 '{}_x'.format(perspective_one_week), # Fares (последний день) Ютэйр                     
                 '{}_y'.format(perspective_one_week) # Fares (последний день) Победа
                ]]
except KeyError:
    otv6 = otv5[['Аэропорт вылета (код)',
                 'Маршрут', 
                 '{} ЮТЭЙР'.format(one_week),
                 '{} ПОБЕДА'.format(one_week),
                 '{} ЮТЭЙР'.format(two_week),
                 '{} ПОБЕДА'.format(two_week), 
                 '{} ЮТЭЙР (направления)'.format(one_week),
                 '{} ЮТЭЙР (направления)'.format(two_week),
                 '{}_x'.format(one_week), # Median Fare Победа
                 '{}_y'.format(one_week), # Median Fare Ютэйр
                 '{}_x'.format(two_week), # Median Fare Победа
                 '{}_y'.format(two_week), # Median Fare Ютэйр            
                 #'{} Процент загрузки'.format(perspective_one_week), # Процент загрузки                      
                 '{} Процент загрузки'.format(perspective_two_week), # Процент загрузки 
                 #'{} Количество свободных мест'.format(perspective_one_week), # Количество свободных мест                      
                 '{} Количество свободных мест'.format(perspective_two_week), # Количество свободных мест                
                 '{}_x'.format(perspective_one_week), # Fares (последний день) Ютэйр                     
                 '{}_y'.format(perspective_one_week)]] # Fares (последний день) Победа

In [200]:
# Initialize example dataframe
np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))], axis=1)
df.iloc[0, 2] = np.nan

def highlight_max(s):
    """Styling function: highlights the maximum in a Series yellow."""
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

def color_negative_red(val):
    """Styling function: apply red font color to negative values."""
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'

# Apply conditional formatting to dataframe
styled = df.style.applymap(color_negative_red).apply(highlight_max)

In [201]:
otv6[:2]

,Аэропорт вылета (код),Маршрут,23 ЮТЭЙР,23 ПОБЕДА,24 ЮТЭЙР,24 ПОБЕДА,23 ЮТЭЙР (направления),24 ЮТЭЙР (направления),23_x,23_y,24_x,24_y,25 Процент загрузки,26 Процент загрузки,25 Количество свободных мест,26 Количество свободных мест,25_x,25_y
0,AAQ,AAQVKO,0.672,0.925,0.865,0.948,0.397411,0.749548,2495.0,3255,3500.0,4855,NaN,NaN,NaN,NaN,3655,3743.0
1,VKO,AAQVKO,0.672,0.925,0.865,0.948,0.943905,0.885038,6005.0,7355,5114.0,7355,0.762616,0.787108,715.0,1080.0,4655,2994.0


In [202]:
otv6['Отклонение'] = otv6[['{} ЮТЭЙР'.format(one_week),
                           '{} ПОБЕДА'.format(one_week)
                          ]].apply(lambda x: x[0]-x[1],axis=1)
otv6
otv6['Отклонение2'] = otv6[['{} ЮТЭЙР'.format(two_week),
                            '{} ПОБЕДА'.format(two_week)
                           ]].apply(lambda x: x[0]-x[1],axis=1)
otv6['Отклонение3'] = otv6[['{}_x'.format(one_week),'{}_y'.format(one_week)]].apply(lambda x: x[1]-x[0],axis=1)
otv6['Отклонение4'] = otv6[['{}_x'.format(two_week),'{}_y'.format(two_week)]].apply(lambda x: x[1]-x[0],axis=1)
otv6['Отклонение5'] = otv6[['{}_x'.format(perspective_one_week),
                            '{}_y'.format(perspective_one_week)]].apply(lambda x: x[0]-x[1],axis=1)

<ipython-input-202-e9a9ec5fa365>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otv6['Отклонение'] = otv6[['{} ЮТЭЙР'.format(one_week),
<ipython-input-202-e9a9ec5fa365>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  otv6['Отклонение2'] = otv6[['{} ЮТЭЙР'.format(two_week),
<ipython-input-202-e9a9ec5fa365>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [203]:
otv6.columns

Index(['Аэропорт вылета (код)', 'Маршрут', '23 ЮТЭЙР', '23 ПОБЕДА', '24 ЮТЭЙР',
       '24 ПОБЕДА', '23 ЮТЭЙР (направления)', '24 ЮТЭЙР (направления)', '23_x',
       '23_y', '24_x', '24_y', '25 Процент загрузки', '26 Процент загрузки',
       '25 Количество свободных мест', '26 Количество свободных мест', '25_x',
       '25_y', 'Отклонение', 'Отклонение2', 'Отклонение3', 'Отклонение4',
       'Отклонение5'],
      dtype='object')

In [204]:
otv7 = otv6.rename(columns = {
    '{}_x'.format(one_week):'{} Median Fare Победа'.format(one_week),
    '{}_x'.format(two_week):'{} Median Fare Победа'.format(two_week),
    '{}_y'.format(one_week):'{} Median Fare Ютэйр'.format(one_week),
    '{}_y'.format(two_week):'{} Median Fare Ютэйр'.format(two_week),
    '{}_x'.format(perspective_one_week):'{} Fares (последний день) Ютэйр'.format(perspective_one_week),
    '{}_y'.format(perspective_one_week):'{} Fares (последний день) Победа'.format(perspective_one_week)
                      })

In [205]:
otv7['index'] = otv7.iloc[:,1]
# df.loc['cobra':'viper', 'max_speed']

In [206]:
otv7.columns

Index(['Аэропорт вылета (код)', 'Маршрут', '23 ЮТЭЙР', '23 ПОБЕДА', '24 ЮТЭЙР',
       '24 ПОБЕДА', '23 ЮТЭЙР (направления)', '24 ЮТЭЙР (направления)',
       '23 Median Fare Победа', '23 Median Fare Ютэйр',
       '24 Median Fare Победа', '24 Median Fare Ютэйр', '25 Процент загрузки',
       '26 Процент загрузки', '25 Количество свободных мест',
       '26 Количество свободных мест', '25 Fares (последний день) Ютэйр',
       '25 Fares (последний день) Победа', 'Отклонение', 'Отклонение2',
       'Отклонение3', 'Отклонение4', 'Отклонение5', 'index'],
      dtype='object')

In [207]:
del otv7['Маршрут']

In [208]:
otv7 = otv7.set_index(['Аэропорт вылета (код)', 'index'])

In [209]:
otv7.rename(columns = {'{} ЮТЭЙР'.format(one_week):'{} ЮТЭЙР (загрузка)'.format(one_week),
                       '{} ПОБЕДА'.format(one_week):'{} ПОБЕДА (загрузка)'.format(one_week), 
                       '{} ЮТЭЙР'.format(two_week):'{} ЮТЭЙР (загрузка)'.format(two_week),
                       '{} ПОБЕДА'.format(two_week):'{} ПОБЕДА (загрузка)'.format(two_week)},
            inplace=True)

In [210]:
one_week

23

In [211]:
try:
    otv7 = otv7[['{} ЮТЭЙР (загрузка)'.format(one_week), 
                 '{} ПОБЕДА (загрузка)'.format(one_week), 
                 'Отклонение', 
                 '{} ЮТЭЙР (загрузка)'.format(two_week), 
                 '{} ПОБЕДА (загрузка)'.format(two_week), 
                 'Отклонение2', 
                 '{} ЮТЭЙР (направления)'.format(one_week), 
                 '{} ЮТЭЙР (направления)'.format(two_week),
                 '{} Median Fare Победа'.format(one_week), 
                 '{} Median Fare Ютэйр'.format(one_week), 
                 'Отклонение3',
                 '{} Median Fare Победа'.format(two_week), 
                 '{} Median Fare Ютэйр'.format(two_week),
                 'Отклонение4', 
                 '{} Процент загрузки'.format(perspective_one_week),
                 '{} Процент загрузки'.format(perspective_two_week), 
                 '{} Количество свободных мест'.format(perspective_one_week),
                 '{} Количество свободных мест'.format(perspective_two_week), 
                 '{} Fares (последний день) Ютэйр'.format(perspective_one_week),
                 '{} Fares (последний день) Победа'.format(perspective_one_week),
                 'Отклонение5']].reset_index()
except KeyError:
    otv7 = otv7[['{} ЮТЭЙР (загрузка)'.format(one_week), 
                 '{} ПОБЕДА (загрузка)'.format(one_week), 
                 'Отклонение', 
                 '{} ЮТЭЙР (загрузка)'.format(two_week), 
                 '{} ПОБЕДА (загрузка)'.format(two_week), 
                 'Отклонение2', 
                 '{} ЮТЭЙР (направления)'.format(one_week), 
                 '{} ЮТЭЙР (направления)'.format(two_week),
                 '{} Median Fare Победа'.format(one_week), 
                 '{} Median Fare Ютэйр'.format(one_week), 
                 'Отклонение3',
                 '{} Median Fare Победа'.format(two_week), 
                 '{} Median Fare Ютэйр'.format(two_week),
                 'Отклонение4', 
                 #'{} Процент загрузки'.format(perspective_one_week),
                 '{} Процент загрузки'.format(perspective_two_week), 
                 #'{} Количество свободных мест'.format(perspective_one_week),
                 '{} Количество свободных мест'.format(perspective_two_week), 
                 '{} Fares (последний день) Ютэйр'.format(perspective_one_week),
                 '{} Fares (последний день) Победа'.format(perspective_one_week),
                 'Отклонение5']].reset_index()

In [212]:
# data =  pd.DataFrame(np.random.randn(5, 3), columns=list('ABC'))
# print (data)
def highlight_cols(s):
    if s > 0:
        color = '#98FB98'
    else:
        color = '#F08080'
    return 'background-color: %s' % color

otv8 = otv7[otv7.columns].style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['Отклонение', 
                                                                                  'Отклонение2', 
                                                                                  'Отклонение3',
                                                                                  'Отклонение4', 
                                                                                  'Отклонение5']])

otv8 = otv8.format({'22 ЮТЭЙР (загрузка)': '{:,.2%}'.format,
                '22 ПОБЕДА (загрузка)': '{:,.2%}'.format, 
                'Отклонение': '{:,.2%}'.format, 
                '23 ЮТЭЙР (загрузка)': '{:,.2%}'.format,
                '23 ПОБЕДА (загрузка)': '{:,.2%}'.format, 
                'Отклонение2': '{:,.2%}'.format, 
                '22 ЮТЭЙР (направления)': '{:,.2%}'.format,
                '23 ЮТЭЙР (направления)': '{:,.2%}'.format, 
                '25 Процент загрузки': '{:,.2%}'.format,
                '22 Median Fare Победа': '{:,.0f}'.format,
                '22 Median Fare Ютэйр': '{:,.0f}'.format, 
                'Отклонение3': '{:,.0f}'.format, 
                '23 Median Fare Победа': '{:,.0f}'.format,
                '23 Median Fare Ютэйр': '{:,.0f}'.format, 
                'Отклонение4': '{:,.0f}'.format, 
                '25 Количество свободных мест': '{:,.0f}'.format,
                '24 Fares (последний день) Ютэйр': '{:,.0f}'.format, 
                '24 Fares (последний день) Победа': '{:,.0f}'.format,
                'Отклонение5': '{:,.0f}'.format}

)

#df.style.format("{:.2%}")

# df.style.format({
#     'var1': '{:,.2f}'.format,
#     'var2': '{:,.2f}'.format,
#     'var3': '{:,.2%}'.format,
# })

In [213]:
otv8.columns

Index(['Аэропорт вылета (код)', 'index', '23 ЮТЭЙР (загрузка)',
       '23 ПОБЕДА (загрузка)', 'Отклонение', '24 ЮТЭЙР (загрузка)',
       '24 ПОБЕДА (загрузка)', 'Отклонение2', '23 ЮТЭЙР (направления)',
       '24 ЮТЭЙР (направления)', '23 Median Fare Победа',
       '23 Median Fare Ютэйр', 'Отклонение3', '24 Median Fare Победа',
       '24 Median Fare Ютэйр', 'Отклонение4', '25 Процент загрузки',
       '26 Процент загрузки', '25 Количество свободных мест',
       '26 Количество свободных мест', '25 Fares (последний день) Ютэйр',
       '25 Fares (последний день) Победа', 'Отклонение5'],
      dtype='object')

In [214]:
#otv8[:5]

In [218]:
%%time
#otv8.to_excel('костыль1.xlsx')

with pd.ExcelWriter(r'AllFutureFlights.xlsx',engine='xlsxwriter') as writer:  
      
    df_res4.to_excel(writer, sheet_name='Flights',header=True, index=False,encoding='utf-8-sig')
    worksheet1=writer.sheets['Flights']
    worksheet1.freeze_panes(1,3)
    result.to_excel(writer, sheet_name='Scenarios',header=True, index=False,encoding='utf-8-sig')
    res_by_controller.to_excel(writer,sheet_name='Controllers',header=True, index=False,encoding='utf-8-sig')
    #новые кривые
    goal_c.to_excel(writer,sheet_name='Curves',header=True, index=True,encoding='utf-8-sig')
    desc.to_excel(writer,sheet_name='Description',header=True, index=True,encoding='utf-8-sig')
    otv8.to_excel(writer, sheet_name='UT and DP',header=True, index=False,encoding='utf-8-sig')

Wall time: 15.4 s


In [219]:
#ws1 = wb.create_sheet("Sheet_A")

In [220]:
%%time
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side

wb = load_workbook('AllFutureFlights.xlsx')
worksheet = wb.worksheets[5]
#wb.create_sheet("UT and DP")
for i in range(1,30):
    worksheet.column_dimensions[get_column_letter(i)].width = 10
row =  list(worksheet.rows)[0]
str_alf = 'ABCDEFGHIJKLMNOPQRSTUVW'
for i in str_alf:
    worksheet['{}1'.format(i)].alignment = Alignment(wrapText=True,horizontal='center',vertical='center')
#     currentCell = ws.cell('{}1'.format(i)) #or currentCell = ws['A1']
#     currentCell.alignment = Alignment(horizontal='center',vertical='center')

thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

for x in range(1,101):
    for y in range(1,24):
        #worksheet.cell(row=x, column=y).number_format = '0.00'
        worksheet.cell(row=x, column=y).border = thin_border
        
worksheet.auto_filter.ref = worksheet.dimensions

# worksheet['A:J'].style = 'Percent'
# worksheet['Q:R'].style = 'Percent'

for x in range(1,101):
    for y in [3,4,5,6,7,8,9,10,17,18]:
        worksheet.cell(row=x, column=y).number_format = '0.00%'
        
# CDEFGHIJ QR ABCDEFGHIJKLMNOPQRSTUVW
# 12345678    12345678901234567890123

wb.save('AllFutureFlights.xlsx')

Wall time: 24.1 s
